<a href="https://colab.research.google.com/github/chandan241/st_tr/blob/main/manual_multiple_select.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
import logging
import time
from kiteconnect import KiteConnect
from collections import deque
from IPython.display import clear_output
# keep last 10 messages
log_buffer = deque(maxlen=10)

logging.basicConfig(level=logging.DEBUG)
api_key="l9k2jvh835bb37nw"
api_secret = "y0qbd7eon5h2odofujubbcudhymiyhvh"
kite = KiteConnect(api_key=api_key)
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=l9k2jvh835bb37nw&v=3'

In [ ]:
request_token = "bk91I4hQKzQ627psOXqYqYI4dLIBC8Lo"

In [ ]:
data = kite.generate_session(request_token, api_secret=api_secret)
kite.set_access_token(data["access_token"])

In [ ]:
# @title Cell 1 (Display the options prices) - Multi-select + Top 3 CE/PE buttons + Refresh
import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

print('**********************CONTROL LOSSES THAN MAKING PROFIT. MARKET WILL GIVE 2ND CHANCE****************************')

INDEX_CONFIG = {
    "NIFTY":  {"segment": "NFO", "name": "NIFTY",  "spot": "NSE:NIFTY 50"},
    "SENSEX": {"segment": "BFO", "name": "SENSEX", "spot": "BSE:SENSEX"},
}

# Global state
checkboxes = {}
ce_list = []            # Ordered list of CE symbols for fast "top 3 CE"
pe_list = []            # Ordered list of PE symbols for fast "top 3 PE"
symbol_meta = {}
current_prefix = None
atm_info = {}
out = widgets.Output()

# --- Widgets ---
index_dropdown = widgets.Dropdown(
    options=[("NIFTY 50", "NIFTY"), ("SENSEX", "SENSEX")],
    value="NIFTY",
    description="Index:",
    style={'description_width': 'initial'}
)

expiry_dropdown = widgets.Dropdown(
    options=[],
    description="Expiry:",
    style={'description_width': 'initial'}
)

btn_top3_ce = widgets.Button(description="Top 3 CE", button_style="info")
btn_top3_pe = widgets.Button(description="Top 3 PE", button_style="info")
btn_clear   = widgets.Button(description="Clear All", button_style="warning")

# NEW: Refresh button
btn_refresh = widgets.Button(description="Refresh", button_style="primary", tooltip="Refresh quotes & ATM")

controls_box = widgets.HBox([index_dropdown, expiry_dropdown, btn_top3_ce, btn_top3_pe, btn_clear, btn_refresh])

display(controls_box, out)

# =========================
# Helper to build checkbox
# =========================
def make_checkbox(sym, strike, label, meta):
    cb = widgets.Checkbox(value=False, description="", layout=widgets.Layout(width="auto"))
    cb.style = {'description_width': '0px'}
    html_label = widgets.HTML(value=label, layout=widgets.Layout(margin="0 0 0 5px"))
    hbox = widgets.HBox([cb, html_label], layout=widgets.Layout(align_items="center"))
    checkboxes[sym] = cb
    symbol_meta[sym] = meta
    return hbox

# =========================
# Load expiries
# =========================
def refresh_expiries(index_key):
    cfg = INDEX_CONFIG[index_key]
    segment = cfg["segment"]
    name = cfg["name"]

    instruments = kite.instruments(segment)

    options = [i for i in instruments if i['name'] == name and i['instrument_type'] in ["CE", "PE"]]
    today = datetime.date.today()
    exps = sorted({i['expiry'] for i in options if i['expiry'] >= today})

    if not exps:
        expiry_dropdown.options = [("No expiries found", None)]
        expiry_dropdown.value = None
        return options, None

    if len(exps) == 1:
        expiry_dropdown.options = [(exps[0].strftime("%d-%b-%Y") + " (Only)", exps[0])]
        expiry_dropdown.value = exps[0]
    else:
        expiry_dropdown.options = [
            ("This Week - " + exps[0].strftime("%d-%b-%Y"), exps[0]),
            ("Next Week - " + exps[1].strftime("%d-%b-%Y"), exps[1]),
        ]
        # preserve existing selection if possible; else default to first
        if expiry_dropdown.value not in exps:
            expiry_dropdown.value = exps[0]

    return options, exps

# =========================
# Render chain
# =========================
def render_chain(index_key, expiry_date, preserve_selection=True):
    global checkboxes, symbol_meta, ce_list, pe_list, current_prefix, atm_info

    # Capture currently selected symbols to preserve (optional)
    previously_selected = set([sym for sym, cb in checkboxes.items() if cb.value]) if preserve_selection else set()

    checkboxes = {}
    symbol_meta = {}
    ce_list = []
    pe_list = []

    cfg = INDEX_CONFIG[index_key]
    segment = cfg["segment"]
    name = cfg["name"]
    spot_symbol = cfg["spot"]
    current_prefix = segment

    instruments = kite.instruments(segment)
    expiry_opts = [
        i for i in instruments
        if i['name'] == name and i['instrument_type'] in ["CE", "PE"] and i['expiry'] == expiry_date
    ]

    if not expiry_opts:
        with out:
            out.clear_output()
            print(f"No options found for {name} on {expiry_date}.")
        return

    # --- Spot & ATM ---
    spot = kite.ltp(spot_symbol)[spot_symbol]["last_price"]
    strikes = sorted({i['strike'] for i in expiry_opts})
    atm_strike = min(strikes, key=lambda s: abs(s - spot))
    atm_info[name] = {"atm_strike": atm_strike, "spot": spot}

    # --- CE & PE OTM selection ---
    ce_otm = sorted([i for i in expiry_opts if i['instrument_type']=="CE" and i['strike'] > atm_strike],
                    key=lambda x: x['strike'])[:10]
    pe_otm = sorted([i for i in expiry_opts if i['instrument_type']=="PE" and i['strike'] < atm_strike],
                    key=lambda x: -x['strike'])[:10]

    ce_list = [opt["tradingsymbol"] for opt in ce_otm]
    pe_list = [opt["tradingsymbol"] for opt in pe_otm]

    symbols = ce_list + pe_list
    quote_data = kite.quote([f"{current_prefix}:{sym}" for sym in symbols]) if symbols else {}

    def get_change(sym):
        key = f"{current_prefix}:{sym}"
        data = quote_data.get(key, {})
        ltp = data.get("last_price")
        prev_close = data.get("ohlc", {}).get("close")

        if ltp is None: return "NA"
        if prev_close:
            pct = ((ltp - prev_close) / prev_close) * 100
            color = "green" if pct >= 0 else "red"
            arrow = "🔺" if pct >= 0 else "🔻"
            return f"{ltp:.2f} (<span style='color:{color};'>{arrow}{pct:.2f}%</span>)"

        return f"{ltp:.2f} (NA)"

    ce_items, pe_items = [], []

    # --- Build CE column ---
    for opt in ce_otm:
        sym = opt["tradingsymbol"]
        label = f"<b>{opt['strike']}</b> | {sym} | {get_change(sym)}"
        meta = {"strike": opt["strike"], "instrument_type": "CE", "expiry": opt["expiry"], "index_name": name}
        cb_row = make_checkbox(sym, opt["strike"], label, meta)
        ce_items.append(cb_row)

    # --- Build PE column ---
    for opt in pe_otm:
        sym = opt["tradingsymbol"]
        label = f"<b>{opt['strike']}</b> | {sym} | {get_change(sym)}"
        meta = {"strike": opt["strike"], "instrument_type": "PE", "expiry": opt["expiry"], "index_name": name}
        cb_row = make_checkbox(sym, opt["strike"], label, meta)
        pe_items.append(cb_row)

    # Restore selections if symbol still present
    if preserve_selection:
        for sym in previously_selected:
            if sym in checkboxes:
                checkboxes[sym].value = True

    atm_label = widgets.HTML(f"<b>{name} ATM Strike: {atm_strike} (Spot: {round(spot,2)})</b>")

    cols = widgets.HBox([
        widgets.VBox([widgets.Label("📈 OTM Calls (CE > ATM)")] + ce_items,
                     layout=widgets.Layout(border="1px solid gray", padding="5px", width="420px")),
        widgets.VBox([widgets.Label("📉 OTM Puts (PE < ATM)")] + pe_items,
                     layout=widgets.Layout(border="1px solid gray", padding="5px", width="420px")),
    ])

    with out:
        out.clear_output()
        display(widgets.VBox([atm_label, cols]))

# =========================
# Button handlers
# =========================
def top3_ce_clicked(b):
    for cb in checkboxes.values():
        cb.value = False  # clear first
    for sym in ce_list[:3]:
        if sym in checkboxes:
            checkboxes[sym].value = True

def top3_pe_clicked(b):
    for cb in checkboxes.values():
        cb.value = False
    for sym in pe_list[:3]:
        if sym in checkboxes:
            checkboxes[sym].value = True

def clear_all_clicked(b):
    for cb in checkboxes.values():
        cb.value = False

# NEW: Refresh handler
def refresh_clicked(b):
    # Option A: fast refresh without touching expiry list
    # render_chain(index_dropdown.value, expiry_dropdown.value, preserve_selection=True)

    # Option B: refresh expiries too (useful around weekly rollovers)
    _, _ = refresh_expiries(index_dropdown.value)
    if expiry_dropdown.value is not None:
        render_chain(index_dropdown.value, expiry_dropdown.value, preserve_selection=True)

btn_top3_ce.on_click(top3_ce_clicked)
btn_top3_pe.on_click(top3_pe_clicked)
btn_clear.on_click(clear_all_clicked)
btn_refresh.on_click(refresh_clicked)  # NEW

# =========================
# Dropdown handlers
# =========================
def on_index_change(change):
    if change['name'] == 'value':
        refresh_expiries(change['new'])
        if expiry_dropdown.value:
            render_chain(change['new'], expiry_dropdown.value)

def on_expiry_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        render_chain(index_dropdown.value, change['new'])

index_dropdown.observe(on_index_change, names='value')
expiry_dropdown.observe(on_expiry_change, names='value')

# Initial load
refresh_expiries(index_dropdown.value)
if expiry_dropdown.value:
    render_chain(index_dropdown.value, expiry_dropdown.value)

# Helper used by Cell-3
def get_selected_symbols():
    return [sym for sym, cb in checkboxes.items() if cb.value]

**********************CONTROL LOSSES THAN MAKING PROFIT. MARKET WILL GIVE 2ND CHANCE****************************


Output()

In [ ]:
# ====== Cell 2 (manual settings) ======
# User-configurable parameters
stoploss_percent = 0.1              # 0.05 = 5% SL (change as per need)
trail_start_percent = 0.1          # Start trailing after 5% profit
trail_step_percent = 0.1            # Each trail step = 5% of buy price
square_off_target_percent = 5.0      # Fully exit trade after 20% profit

# Derived / extra config
enable_square_off = True            # Enable auto square-off when target achieved
log_folder = "/content/TradingLog"  # Colab working path where logs will be saved
# ===== New User Parameters =====
buffer_pct = 0.03                     # 2% safety-buffer while checking available funds
chunk_retry = 2                       # Retry attempts for chunked orders (SL / exit etc.)

In [ ]:
# @title
# ============================
# CELL 3 v2 — Fast + Round-Robin + Freeze-based max qty/order
# ============================

import os
import time
import signal
import sys
from datetime import datetime, date
from zoneinfo import ZoneInfo
from IPython.display import clear_output

# ----------------------------
# User confirmation + required vars
# ----------------------------
confirm = input("Every trade you take will take a toll on you health. So trade judiciously. You have to take only few CORRECT decisions, Is it one among them? (Y/N): ").strip().lower()
if confirm not in ("y", "yes"):
    raise SystemExit(" Aborted by user.")

required_vars = [
    "stoploss_percent",
    "trail_start_percent",
    "trail_step_percent",
    "square_off_target_percent",
    "log_folder",
]
missing = [v for v in required_vars if v not in globals() or globals()[v] is None]
if missing:
    raise NameError(f"Missing required vars: {missing}. Please define them before running Cell3 v2.")

# Optional settings with defaults
buffer_pct     = globals().get("buffer_pct", 0.02)         # safety buffer for fund usage
chunk_retry    = globals().get("chunk_retry", 2)           # retries for cancel/modify
exchange_prefix = globals().get("current_prefix", "NFO")   # set by selector cell
enable_square_off = globals().get("enable_square_off", True)

# Freeze limit per order (units), user can override globally
DEFAULT_MAX_FREEZE_QTY = globals().get("max_freeze_qty_per_order", 1800 if exchange_prefix == "NFO" else 600)

# Speed controls
FAST_MODE     = True
API_THROTTLE  = 0.08 if FAST_MODE else 0.60   # small delay between API calls
POLL_INTERVAL = 0.35 if FAST_MODE else 0.80   # how often to poll kite.orders()
FILL_TIMEOUT  = 12   if FAST_MODE else 30     # max seconds to wait for fills
MONITOR_SLEEP = 2.0  if FAST_MODE else 3.0    # monitor frequency

# ----------------------------
# Logging (IST timestamps)
# ----------------------------
os.makedirs(log_folder, exist_ok=True)
log_file_path = os.path.join(log_folder, date.today().isoformat() + ".txt")

def write_log(line: str):
    ts = datetime.now(ZoneInfo("Asia/Kolkata")).isoformat(timespec="seconds")
    with open(log_file_path, "a") as f:
        f.write(f"{ts}\n{line}\n")

write_log("=== New trading session started (Cell3 v2) ===")

# ----------------------------
# Kite wrappers
# ----------------------------
def kite_available_cash_once():
    try:
        m = kite.margins()
        return float(m["equity"]["available"]["live_balance"])
    except Exception as e:
        write_log(f"Could not fetch kite margins: {e}")
        return 0.0

def key_for(sym: str) -> str:
    return f"{exchange_prefix}:{sym}"

def get_ltps(symbols):
    """
    Batch LTP fetch: returns {sym: ltp}
    """
    if not symbols:
        return {}
    keys = [key_for(s) for s in symbols]
    try:
        data = kite.ltp(keys)
        out = {}
        for s in symbols:
            out[s] = data.get(key_for(s), {}).get("last_price")
        return out
    except Exception as e:
        write_log(f"get_ltps failed: {e}")
        return {s: None for s in symbols}

def place_market_buy(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_BUY,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_market_sell(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_sl_sell(sym, qty, sl_price):
    # NOTE: if your account/exchange requires trigger/limit gap, add a small gap here.
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_SL,
        price=float(sl_price),
        trigger_price=float(sl_price)
    )

# ----------------------------
# Orders polling (batch, fast)
# ----------------------------
TERMINAL = {"COMPLETE", "REJECTED", "CANCELLED"}

def fetch_orders_map():
    try:
        all_orders = kite.orders()
        return {o.get("order_id"): o for o in all_orders if o.get("order_id")}
    except Exception as e:
        write_log(f"fetch_orders_map failed: {e}")
        return {}

def wait_for_orders(order_ids, timeout_sec=12, poll_interval=0.35):
    order_ids = [oid for oid in order_ids if oid]
    if not order_ids:
        return {}

    start = time.time()
    last_seen = {}

    while time.time() - start < timeout_sec:
        omap = fetch_orders_map()
        for oid in order_ids:
            if oid in omap:
                last_seen[oid] = omap[oid]

        done = sum(1 for oid in order_ids if (last_seen.get(oid, {}) or {}).get("status") in TERMINAL)
        if done == len(order_ids):
            return last_seen

        time.sleep(poll_interval)

    write_log(f"wait_for_orders timeout after {timeout_sec}s; returning partial statuses.")
    return last_seen

# ----------------------------
# Lot size: prefer instrument lookup, fallback to heuristic
# ----------------------------
_lot_size_cache = {}

def build_lot_size_cache(selected_syms):
    """
    Build a dict {tradingsymbol: lot_size} by scanning kite.instruments(segment) once.
    Falls back if not found.
    """
    global _lot_size_cache
    try:
        inst = kite.instruments(exchange_prefix)  # segment is NFO/BFO etc.
        need = set(selected_syms)
        cache = {}
        for i in inst:
            ts = i.get("tradingsymbol")
            if ts in need:
                ls = i.get("lot_size") or i.get("market_lot") or i.get("lot_size")
                if ls:
                    cache[ts] = int(ls)
        _lot_size_cache = cache
        write_log(f"Lot size cache built for {len(cache)}/{len(selected_syms)} symbols.")
    except Exception as e:
        write_log(f"Lot size cache build failed: {e}")
        _lot_size_cache = {}

def infer_lot_size(sym: str) -> int:
    """
    Use cached instrument lot size if available.
    Else fallback (NIFTY=75, SENSEX=20) heuristic.
    """
    if sym in _lot_size_cache:
        return int(_lot_size_cache[sym])

    # Fallback heuristic (kept for compatibility)
    if "SENSEX" in sym.upper() or exchange_prefix == "BFO":
        return 20
    return 75

# ----------------------------
# Chunking utilities (freeze-based max qty/order)
# ----------------------------
def max_units_per_order(lot_size: int) -> int:
    """
    Max units per single order based on freeze limit, aligned to lot size.
    """
    max_units = int(DEFAULT_MAX_FREEZE_QTY)
    lots = max(1, max_units // lot_size)
    return int(lots * lot_size)

def aligned_affordable_chunk(remaining_funds, ltp, lot_size, max_units, buffer_pct):
    """
    Largest chunk aligned to lot size, capped by freeze-based max_units.
    """
    if not ltp or ltp <= 0:
        return 0
    affordable_units = int(remaining_funds // (ltp * (1 + buffer_pct)))
    affordable_lots = affordable_units // lot_size
    if affordable_lots <= 0:
        return 0
    units = affordable_lots * lot_size
    return int(min(units, max_units))

def split_units_to_chunks(total_units, max_units, lot_size):
    """
    If you need chunk splitting (sells / SL), align each chunk to lot size.
    """
    chunks = []
    remaining = int(total_units)
    if max_units < lot_size:
        max_units = lot_size

    while remaining > 0:
        take = min(remaining, max_units)
        rem_mod = take % lot_size
        if rem_mod != 0:
            take -= rem_mod
            if take <= 0:
                take = lot_size
        chunks.append(int(take))
        remaining -= take
    return chunks

# ----------------------------
# Registries
# ----------------------------
open_positions_registry = {}  # {sym: {"qty": int, "sl_order_ids":[...]}}
positions = {}                # {sym: state dict}

# ----------------------------
# SL helpers (split SL)
# ----------------------------
def place_split_sl_orders(sym, total_units, sl_price, lot_size):
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(total_units, max_u, lot_size)
    placed = []
    for chunk in chunks:
        try:
            oid = place_sl_sell(sym, chunk, sl_price)
            placed.append({"order_id": oid, "qty": chunk, "price": float(sl_price)})
            time.sleep(API_THROTTLE)
        except Exception as e:
            write_log(f"ERROR placing SL chunk {sym} qty={chunk}: {e}")
    return placed

def cancel_order_fast(order_id):
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)
        return True
    except Exception as e:
        # ignore common states (already complete/cancelled)
        write_log(f"cancel_order_fast ignore {order_id}: {e}")
        return False

def cancel_sl_fast(order_ids):
    for oid in order_ids or []:
        if oid:
            cancel_order_fast(oid)
            time.sleep(API_THROTTLE)

def modify_sl_order_with_retry(order_id, new_price, symbol):
    for attempt in range(1, chunk_retry + 1):
        try:
            kite.modify_order(
                variety=kite.VARIETY_REGULAR,
                order_id=order_id,
                price=float(new_price),
                trigger_price=float(new_price)
            )
            return True
        except Exception as e:
            write_log(f"{symbol} ERROR modify SL {order_id} -> {new_price} attempt={attempt}: {e}")
            time.sleep(0.25 * attempt if FAST_MODE else 0.8 * (2 ** (attempt - 1)))
    return False

# ----------------------------
# Fast safe_exit: cancel SLs then market sell in freeze-aligned chunks
# ----------------------------
def split_market_sell(sym, total_units, lot_size):
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(total_units, max_u, lot_size)
    sell_oids = []
    for chunk in chunks:
        try:
            oid = place_market_sell(sym, chunk)
            sell_oids.append(oid)
            time.sleep(API_THROTTLE)
        except Exception as e:
            write_log(f"ERROR market sell {sym} chunk={chunk}: {e}")
    # optional: wait briefly for sells to complete
    wait_for_orders(sell_oids, timeout_sec=FILL_TIMEOUT, poll_interval=POLL_INTERVAL)
    return sell_oids

def safe_exit(sym, reason="manual"):
    st = positions.get(sym, {}) or {}
    qty = int(st.get("quantity", 0) or open_positions_registry.get(sym, {}).get("qty", 0) or 0)
    if qty <= 0:
        return

    write_log(f"[safe_exit] {sym} qty={qty} reason={reason}")

    # Cancel all SL orders first to prevent accidental short
    sl_ids = []
    sl_ids.extend(open_positions_registry.get(sym, {}).get("sl_order_ids", []) or [])
    sl_ids.extend([e.get("order_id") for e in (st.get("sl_orders") or []) if e.get("order_id")])
    cancel_sl_fast(list(set(sl_ids)))

    # Sell in freeze-aligned chunks
    lot_size = int(st.get("lot_size") or infer_lot_size(sym))
    split_market_sell(sym, qty, lot_size)

    # Cleanup
    open_positions_registry.pop(sym, None)
    positions.pop(sym, None)
    write_log(f"[safe_exit] Completed cleanup for {sym}")

# ----------------------------
# Emergency handler: FAST exit all
# ----------------------------
def emergency_exit_handler(signum, frame):
    print("\n EMERGENCY STOP — FAST EXIT ALL...")
    write_log("EMERGENCY STOP received. FAST EXIT ALL started.")

    # Cancel all SL ids across all symbols first
    all_sl = set()
    for sym, reg in list(open_positions_registry.items()):
        for oid in (reg.get("sl_order_ids") or []):
            if oid:
                all_sl.add(oid)
    for sym, st in list(positions.items()):
        for e in (st.get("sl_orders") or []):
            oid = e.get("order_id")
            if oid:
                all_sl.add(oid)

    cancel_sl_fast(list(all_sl))

    # Then sell all
    for sym in list(open_positions_registry.keys()):
        try:
            safe_exit(sym, reason="emergency")
        except Exception as e:
            write_log(f"Emergency safe_exit failed for {sym}: {e}")

    write_log("EMERGENCY EXIT COMPLETE")
    print(" Emergency exit finished. Exiting notebook.")
    sys.exit(0)

signal.signal(signal.SIGINT, emergency_exit_handler)

# ----------------------------
# Selection validation (from selector cell)
# ----------------------------
selected = get_selected_symbols()
if not selected:
    raise SystemExit(" No symbols selected. Select in chain UI and re-run.")

if "symbol_meta" not in globals() or not symbol_meta:
    raise SystemExit(" symbol_meta missing. Re-render selector chain and re-select.")

first_meta = symbol_meta[selected[0]]
index_name = first_meta.get("index_name")

atm_data = atm_info.get(index_name, {}) if "atm_info" in globals() else {}
atm_strike = atm_data.get("atm_strike", None)

def strike_distance(sym):
    if atm_strike is None:
        return 0
    return abs((symbol_meta.get(sym, {}) or {}).get("strike", 10**18) - atm_strike)

selected_sorted = sorted(selected, key=strike_distance)
write_log(f"Selected symbols (sorted): {selected_sorted}")

print("Selected symbols:")
for s in selected_sorted:
    m = symbol_meta.get(s, {}) or {}
    print(f" - {s} strike={m.get('strike')} type={m.get('instrument_type')}")

# Build lot size cache once (fast lookups later)
build_lot_size_cache(selected_sorted)

# ----------------------------
# Funds input
# ----------------------------
initial_cash = kite_available_cash_once()
print(f"Available Live Balance (once) = ₹{initial_cash:.2f}")
user_input = input(f"Enter fund to use (press Enter to use {initial_cash:.2f}): ").strip()
fund = float(user_input) if user_input != "" else initial_cash
write_log(f"Fund allocated: ₹{fund:.2f}")
print(f"Fund set to ₹{fund:.2f}")

remaining_funds = fund * (1 - buffer_pct)
write_log(f"Usable funds after buffer ({buffer_pct*100:.1f}%): ₹{remaining_funds:.2f}")

# ----------------------------
# ROUND-ROBIN ALLOCATION
# ----------------------------
print("\nStarting ROUND-ROBIN allocation (freeze-based max qty/order)...")
write_log("Starting ROUND-ROBIN allocation...")

placed_orders = []  # list of {sym, oid, req_qty, est_ltp}
progress = True

# Pre-compute per symbol constraints
lot_map = {s: infer_lot_size(s) for s in selected_sorted}
maxu_map = {s: max_units_per_order(lot_map[s]) for s in selected_sorted}

# ltp cache for estimation
ltp_cache = {}

while remaining_funds > 0 and progress:
    progress = False

    # One round: each symbol gets at most one order
    ltps = get_ltps(selected_sorted)  # batch
    for sym in selected_sorted:
        ltp = ltps.get(sym) or ltp_cache.get(sym)
        if not ltp:
            continue
        ltp_cache[sym] = ltp

        lot_size = lot_map[sym]
        max_units = maxu_map[sym]

        chunk = aligned_affordable_chunk(remaining_funds, ltp, lot_size, max_units, buffer_pct)
        if chunk <= 0:
            continue

        try:
            oid = place_market_buy(sym, chunk)
            placed_orders.append({"sym": sym, "order_id": oid, "req_qty": int(chunk), "est_ltp": float(ltp)})
            write_log(f"RR BUY placed {sym} qty={chunk} oid={oid} est_ltp={ltp}")

            # reserve conservatively using ltp
            remaining_funds = max(0.0, remaining_funds - (ltp * chunk))
            progress = True
            time.sleep(API_THROTTLE)
        except Exception as e:
            write_log(f"RR BUY failed {sym} qty={chunk}: {e}")

if not placed_orders:
    write_log("No purchases made. Exiting.")
    raise SystemExit(" No purchases were made with provided funds / available LTPs. Exiting.")

# Batch wait for fills
order_ids = [x["order_id"] for x in placed_orders]
status_map = wait_for_orders(order_ids, timeout_sec=FILL_TIMEOUT, poll_interval=POLL_INTERVAL)

# Aggregate fills per symbol
fills_by_sym = {}
for rec in placed_orders:
    sym = rec["sym"]
    oid = rec["order_id"]
    od = status_map.get(oid, {}) or {}
    st = od.get("status")

    if st == "REJECTED":
        write_log(f"Order REJECTED: {sym} oid={oid}")
        continue

    filled_qty = int(od.get("filled_quantity") or od.get("quantity") or rec["req_qty"])
    avg_px = float(od.get("average_price") or rec["est_ltp"] or 0.0)
    if filled_qty <= 0:
        continue

    agg = fills_by_sym.get(sym, {"qty": 0, "w_sum": 0.0, "orders": []})
    agg["qty"] += filled_qty
    agg["w_sum"] += avg_px * filled_qty
    agg["orders"].append({"order_id": oid, "qty": filled_qty, "avg_price": avg_px, "status": st})
    fills_by_sym[sym] = agg

# Place SL per symbol after aggregation
for sym, agg in fills_by_sym.items():
    qty = agg["qty"]
    avg_buy = agg["w_sum"] / qty if qty > 0 else (ltp_cache.get(sym) or 0.0)

    lot_size = lot_map[sym]
    current_sl = round(avg_buy * (1 - stoploss_percent), 1)

    sl_entries = []
    try:
        sl_entries = place_split_sl_orders(sym, qty, current_sl, lot_size)
        write_log(f"SL placed for {sym} chunks={len(sl_entries)} at {current_sl}")
    except Exception as e:
        write_log(f"ERROR placing SL for {sym}: {e}")
        print(f" SL placement failed for {sym}: {e}")

    sl_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]
    open_positions_registry[sym] = {"qty": int(qty), "sl_order_ids": sl_ids}

    positions[sym] = {
        "quantity": int(qty),
        "avg_buy_price": float(avg_buy),
        "sl_orders": sl_entries,
        "current_sl": float(current_sl),
        "lot_size": int(lot_size),
        "sub_orders": agg["orders"],
    }

    print(f"Bought {sym} qty={qty} avg={avg_buy:.2f} SL={current_sl}")
    write_log(f"AGG BUY {sym} qty={qty} avg={avg_buy} SL={current_sl} sl_chunks={len(sl_entries)}")

if not positions:
    write_log("No positions to monitor after allocation. Exiting.")
    raise SystemExit(" No positions created. Exiting.")

write_log(f"Allocation complete. Remaining usable funds (est) ₹{remaining_funds:.2f}")
print("\nAllocation complete. Entering monitoring phase.")
print("Monitoring... Press Ctrl+C for EMERGENCY STOP (fast exit).")

# ----------------------------
# Monitoring loop (faster + safer)
# ----------------------------
log_buffer = []
try:
    while True:
        if not positions:
            write_log("All positions closed. Exiting monitor.")
            break

        # Fetch kite positions ONCE per cycle
        pos_data = {}
        try:
            p = kite.positions()
            pos_data = {(x.get("exchange"), x.get("tradingsymbol")): x.get("quantity", 0)
                        for x in (p.get("net", []) or [])}
        except Exception as e:
            write_log(f"positions() fetch failed: {e}")
            pos_data = {}

        syms = list(positions.keys())
        ltps = get_ltps(syms)  # batch LTPs

        # Orders map once per cycle (for SL status checks)
        orders_map = fetch_orders_map()

        # Display area
        clear_output(wait=True)
        print("=== Trading monitor (Cell3 v2) ===")
        print(f"Remaining usable funds (est): ₹{remaining_funds:.2f}")
        print("Tracked symbols:", syms)
        print("-" * 50)

        # Square-off trigger flag
        square_off_now = False

        for sym in list(positions.keys()):
            st = positions.get(sym, {}) or {}
            qty_open = pos_data.get((exchange_prefix, sym), None)

            # If broker reports closed, stop tracking
            if qty_open is not None and qty_open <= 0:
                write_log(f"{sym} closed (broker qty=0). Removing from tracking.")
                positions.pop(sym, None)
                open_positions_registry.pop(sym, None)
                continue

            ltp = ltps.get(sym)
            if not ltp:
                continue

            avg_buy = st["avg_buy_price"]
            profit_pct = (ltp - avg_buy) / avg_buy if avg_buy else 0.0

            msg = f"{sym} LTP:{ltp:.2f} Pct:{profit_pct*100:.2f}% SL:{st['current_sl']}"
            log_buffer.append(msg)
            log_buffer = log_buffer[-20:]
            print(msg)

            # --- SL miss protection (improved for multi-SL chunks) ---
            sl_entries = st.get("sl_orders", []) or []
            sl_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]

            if sl_ids and ltp <= st["current_sl"]:
                # Determine if *any* SL is actually in terminal COMPLETE
                statuses = []
                for oid in sl_ids:
                    od = orders_map.get(oid, {}) or {}
                    statuses.append(od.get("status"))

                any_complete = any(s == "COMPLETE" for s in statuses if s)
                any_openish = any(s in ("OPEN", "TRIGGER PENDING", None) for s in statuses)

                # If price crossed SL and nothing completed, force exit fast
                if (not any_complete) and any_openish:
                    write_log(f"SL miss detected for {sym} (ltp<=SL). Forcing safe_exit.")
                    safe_exit(sym, reason="sl_miss_force_exit")
                    continue

            # --- Trailing SL ---
            if profit_pct >= trail_start_percent:
                trail_step_value = avg_buy * trail_step_percent
                new_sl = round(ltp - trail_step_value, 1)
                if new_sl > st["current_sl"]:
                    # modify all SL orders
                    ok_count = 0
                    for e in sl_entries:
                        oid = e.get("order_id")
                        if not oid:
                            continue
                        if modify_sl_order_with_retry(oid, new_sl, sym):
                            ok_count += 1
                            e["price"] = float(new_sl)
                        time.sleep(API_THROTTLE)
                    st["current_sl"] = float(new_sl)
                    write_log(f"Trailing SL updated {sym} -> {new_sl} (modified {ok_count}/{len(sl_entries)})")

            # --- Square-off target ---
            if enable_square_off and profit_pct >= square_off_target_percent:
                square_off_now = True

        # Global square-off: exit all open positions
        if enable_square_off and square_off_now:
            write_log("Square-off target reached for at least one symbol. Exiting all positions.")
            print("\n Square-off target reached. Exiting ALL positions...")
            for sym in list(positions.keys()):
                safe_exit(sym, reason="square_off")
            continue

        time.sleep(MONITOR_SLEEP)

except KeyboardInterrupt:
    write_log("KeyboardInterrupt caught; triggering emergency exit handler.")
    emergency_exit_handler(None, None)
finally:
    write_log(" Strategy stopped safely (Cell3 v2).")
    print(" Strategy stopped safely (Cell3 v2).")

KeyboardInterrupt: Interrupted by user

In [ ]:
# @title
# ============================================
# CELL 4 — Breakout Auto Trader (09:50–15:30)
# Breakout on underlying spot, trade selected options
# ============================================

import os, time, sys, signal
from datetime import datetime, date, timedelta, time as dtime
from zoneinfo import ZoneInfo
from collections import deque
from IPython.display import clear_output

from datetime import time as dtime


# Breakout reference window start.
# If None -> market open is used (09:15 IST)
REFERENCE_START_TIME = None   # e.g., dtime(9, 50) OR None

MARKET_OPEN_TIME = dtime(9, 15)  # default market open

# ----------------------------
# REQUIRED: must exist (same as Cell3 expects)
# ----------------------------
required_vars = [
    "stoploss_percent",
    "trail_start_percent",
    "trail_step_percent",
    "square_off_target_percent",
    "log_folder",
]
missing = [v for v in required_vars if v not in globals() or globals()[v] is None]
if missing:
    raise NameError(f"Missing required vars: {missing}. Define before running Cell4.")

# ----------------------------
# CONFIG (edit as needed)
# ----------------------------
IST = ZoneInfo("Asia/Kolkata")

START_TIME = dtime(9, 50)
END_TIME   = dtime(15, 30)
FORCE_EXIT_TIME = dtime(15, 29, 50)

EXCLUDE_LAST_MINUTES = 3

# ✅ BUFFER IN PERCENTAGE (as you confirmed)
BREAKOUT_BUFFER_PCT = 0.0002   # 0.02% default (change to what you want)

MAX_TRADES_PER_DAY = 3
MAX_DAILY_LOSS_PCT = 0.20

# ✅ Paper trade option
PAPER_TRADE = True  # set False for LIVE

# Speed
SPOT_POLL_SEC = 1.0
MONITOR_SLEEP_SEC = 1.5

# Debounce / re-entry control (recommended)
# If True: after a trade closes, strategy will wait until spot returns "inside" the range before taking a new breakout.
REARM_REQUIRED = True

# Fund usage buffer
buffer_pct = globals().get("buffer_pct", 0.02)

# Exchange prefix from Cell2 (NFO/BFO)
exchange_prefix = globals().get("current_prefix", "NFO")

# Enable target square-off (from your existing style)
enable_square_off = globals().get("enable_square_off", True)

# Freeze default (can override globally as in Cell3)
DEFAULT_MAX_FREEZE_QTY = globals().get(
    "max_freeze_qty_per_order",
    1800 if exchange_prefix == "NFO" else 600
)


# ----------------------------
# Trade summary tracking
# ----------------------------
trade_history = []
# each entry:
# {
#   "symbol": str,
#   "direction": "UP"/"DOWN",
#   "qty": int,
#   "avg_buy": float,
#   "exit_price": float,
#   "pnl": float,
#   "reason": str,
# }
def write_log(line: str):
    ts = datetime.now(IST).isoformat(timespec="seconds")
    with open(log_file_path, "a") as f:
        f.write(f"{ts} | {line}\n")
# ----------------------------
# Logging
# ----------------------------
os.makedirs(log_folder, exist_ok=True)
log_file_path = os.path.join(log_folder, date.today().isoformat() + "_breakout.txt")
def now_ist():
    return datetime.now(IST)
# ----------------------------
# Selection from Cell2
# ----------------------------
selected = get_selected_symbols()
if not selected:
    raise SystemExit("No symbols selected. Select in chain UI (Cell2) and re-run Cell4.")

if "symbol_meta" not in globals() or not symbol_meta:
    raise SystemExit("symbol_meta missing. Re-render selector chain (Cell2) and re-select.")

selected_ce = [s for s in selected if (symbol_meta.get(s, {}) or {}).get("instrument_type") == "CE"]
selected_pe = [s for s in selected if (symbol_meta.get(s, {}) or {}).get("instrument_type") == "PE"]
if not selected_ce or not selected_pe:
    raise SystemExit("Need at least ONE CE and ONE PE selected for UP/DOWN breakouts.")

first_meta = symbol_meta[selected[0]]
index_name = first_meta.get("index_name")

# underlying spot symbol from Cell2 INDEX_CONFIG
spot_symbol = None
if "INDEX_CONFIG" in globals():
    for _, cfg in INDEX_CONFIG.items():
        if cfg.get("name") == index_name:
            spot_symbol = cfg.get("spot")
            break
if not spot_symbol:
    raise SystemExit("Could not determine underlying spot symbol from INDEX_CONFIG. Re-run Cell2.")
def reference_start_dt():
    t0 = REFERENCE_START_TIME if REFERENCE_START_TIME is not None else MARKET_OPEN_TIME
    return datetime.combine(date.today(), t0, tzinfo=IST)
def find_instrument_token_for_spot(spot_symbol: str):
    """
    spot_symbol like 'NSE:NIFTY 50' or 'BSE:SENSEX'
    returns instrument_token if found, else None
    """
    try:
        ex, ts = spot_symbol.split(":")
        inst = kite.instruments(ex)  # NSE or BSE
        for i in inst:
            if i.get("tradingsymbol") == ts:
                return i.get("instrument_token")
    except Exception as e:
        write_log(f"Token lookup failed for {spot_symbol}: {e}")
    return None


def seed_spot_samples_from_history(start_dt: datetime, end_dt: datetime, interval="minute"):
    """
    Seeds spot_samples using historical candles from start_dt to end_dt
    interval='minute' keeps it light and good enough for breakout.
    """
    token = find_instrument_token_for_spot(spot_symbol)
    if not token:
        write_log("Could not seed history: instrument_token not found. Will rely on live samples only.")
        return

    try:
        candles = kite.historical_data(token, start_dt, end_dt, interval, continuous=False, oi=False)
        # candles entries have keys: date, open, high, low, close, volume
        added = 0
        for c in candles:
            ts = c.get("date")
            # Use close as representative price for that minute
            px = c.get("close")
            if ts and px:
                spot_samples.append((ts.astimezone(IST), float(px)))
                added += 1
        write_log(f"Seeded {added} historical samples from {start_dt.time()} to {end_dt.time()} ({interval}).")
    except Exception as e:
        write_log(f"Seeding history failed: {e}")
# Seed history from reference start to now (helps if you run late)
hist_start = reference_start_dt()
hist_end = now_ist()

# If run very early (before ref start), skip seeding
if hist_end > hist_start:
    seed_spot_samples_from_history(hist_start, hist_end, interval="minute")



write_log("=== Breakout session started (Cell4) ===")



def key_for(sym: str) -> str:
    return f"{exchange_prefix}:{sym}"

# ----------------------------
# Kite helpers
# ----------------------------
def kite_available_cash_once():
    try:
        m = kite.margins()
        return float(m["equity"]["available"]["live_balance"])
    except Exception as e:
        write_log(f"Could not fetch kite margins: {e}")
        return 0.0

def get_ltp_any(full_symbol: str):
    try:
        d = kite.ltp([full_symbol])
        return d.get(full_symbol, {}).get("last_price")
    except Exception as e:
        write_log(f"LTP fetch failed {full_symbol}: {e}")
        return None

def get_ltps(symbols):
    if not symbols:
        return {}
    keys = [key_for(s) for s in symbols]
    try:
        data = kite.ltp(keys)
        return {s: data.get(key_for(s), {}).get("last_price") for s in symbols}
    except Exception as e:
        write_log(f"get_ltps failed: {e}")
        return {s: None for s in symbols}

TERMINAL = {"COMPLETE", "REJECTED", "CANCELLED"}

def fetch_orders_map():
    try:
        all_orders = kite.orders()
        return {o.get("order_id"): o for o in all_orders if o.get("order_id")}
    except Exception as e:
        write_log(f"fetch_orders_map failed: {e}")
        return {}

def wait_for_orders(order_ids, timeout_sec=15, poll_interval=0.35):
    order_ids = [oid for oid in order_ids if oid]
    if not order_ids:
        return {}
    start = time.time()
    last_seen = {}
    while time.time() - start < timeout_sec:
        omap = fetch_orders_map()
        for oid in order_ids:
            if oid in omap:
                last_seen[oid] = omap[oid]
        done = sum(1 for oid in order_ids if (last_seen.get(oid, {}) or {}).get("status") in TERMINAL)
        if done == len(order_ids):
            return last_seen
        time.sleep(poll_interval)
    write_log(f"wait_for_orders timeout after {timeout_sec}s; returning partial.")
    return last_seen

# ----------------------------
# Order wrappers (LIVE)
# ----------------------------
def place_market_buy(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_BUY,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_market_sell(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_sl_sell(sym, qty, sl_price):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_SL,
        price=float(sl_price),
        trigger_price=float(sl_price)
    )

def cancel_order_fast(order_id):
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)
        return True
    except Exception as e:
        write_log(f"cancel_order_fast ignore {order_id}: {e}")
        return False

def modify_sl_order(order_id, new_price):
    try:
        kite.modify_order(
            variety=kite.VARIETY_REGULAR,
            order_id=order_id,
            price=float(new_price),
            trigger_price=float(new_price)
        )
        return True
    except Exception as e:
        write_log(f"modify_sl_order failed {order_id} -> {new_price}: {e}")
        return False

# ----------------------------
# Freeze/lot utilities
# ----------------------------
def infer_lot_size(sym: str) -> int:
    if "SENSEX" in sym.upper() or exchange_prefix == "BFO":
        return 20
    return 65

def max_units_per_order(lot_size: int) -> int:
    max_units = int(DEFAULT_MAX_FREEZE_QTY)
    lots = max(1, max_units // lot_size)
    return int(lots * lot_size)

def aligned_affordable_chunk(remaining_funds, ltp, lot_size, max_units, buffer_pct):
    if not ltp or ltp <= 0:
        return 0
    affordable_units = int(remaining_funds // (ltp * (1 + buffer_pct)))
    affordable_lots = affordable_units // lot_size
    if affordable_lots <= 0:
        return 0
    units = affordable_lots * lot_size
    return int(min(units, max_units))

def split_units_to_chunks(total_units, max_units, lot_size):
    chunks = []
    remaining = int(total_units)
    if max_units < lot_size:
        max_units = lot_size
    while remaining > 0:
        take = min(remaining, max_units)
        rem_mod = take % lot_size
        if rem_mod != 0:
            take -= rem_mod
        if take <= 0:
            take = lot_size
        chunks.append(int(take))
        remaining -= take
    return chunks

def place_split_sl_orders(sym, total_units, sl_price, lot_size):
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(total_units, max_u, lot_size)
    placed = []
    for chunk in chunks:
        try:
            oid = place_sl_sell(sym, chunk, sl_price)
            placed.append({"order_id": oid, "qty": int(chunk), "price": float(sl_price)})
            time.sleep(0.08)
        except Exception as e:
            write_log(f"ERROR placing SL chunk {sym} qty={chunk}: {e}")
    return placed

def safe_exit_real(sym, qty, sl_order_ids, lot_size, reason="exit"):
    write_log(f"[safe_exit_real] {sym} qty={qty} reason={reason}")
    # cancel SLs
    for oid in (sl_order_ids or []):
        if oid:
            cancel_order_fast(oid)
            time.sleep(0.08)
    # sell chunks
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(qty, max_u, lot_size)
    for chunk in chunks:
        try:
            place_market_sell(sym, chunk)
            time.sleep(0.08)
        except Exception as e:
            write_log(f"ERROR market sell {sym} chunk={chunk}: {e}")

# ----------------------------
# Paper broker simulator
# ----------------------------
class PaperBroker:
    def __init__(self):
        self._oid = 100000
        self.positions = {}  # sym -> {qty, w_sum, avg, sl, open, realized_pnl}
    def _next_oid(self):
        self._oid += 1
        return str(self._oid)
    def buy_market(self, sym, qty, ltp):
        oid = self._next_oid()
        pos = self.positions.get(sym, {"qty": 0, "w_sum": 0.0, "open": False})
        pos["qty"] += int(qty)
        pos["w_sum"] += float(ltp) * int(qty)
        pos["avg"] = pos["w_sum"] / pos["qty"]
        pos["open"] = True
        self.positions[sym] = pos
        return oid, pos["avg"]
    def set_sl(self, sym, sl_price):
        if sym in self.positions:
            self.positions[sym]["sl"] = float(sl_price)
    def modify_sl(self, sym, new_sl):
        if sym in self.positions:
            self.positions[sym]["sl"] = float(new_sl)
    def exit_market(self, sym, ltp, reason="exit"):
        if sym not in self.positions or not self.positions[sym].get("open"):
            return 0.0
        pos = self.positions[sym]
        qty = int(pos["qty"])
        avg = float(pos["avg"])
        pnl = (float(ltp) - avg) * qty
        pos["open"] = False
        pos["realized_pnl"] = pnl
        pos["exit_reason"] = reason
        return pnl

paper = PaperBroker()



atm_data = (atm_info.get(index_name, {}) if "atm_info" in globals() else {}) or {}
atm_strike = atm_data.get("atm_strike")

def strike_distance(sym):
    if atm_strike is None:
        return 10**18
    return abs((symbol_meta.get(sym, {}) or {}).get("strike", 10**18) - atm_strike)

# ✅ nearest selected CE/PE (as you confirmed)
trade_ce = sorted(selected_ce, key=strike_distance)[0]
trade_pe = sorted(selected_pe, key=strike_distance)[0]

write_log(f"Configured CE={trade_ce} PE={trade_pe} Underlying={spot_symbol} ATM={atm_strike}")
print("Breakout strategy configured:")
print(f" - Underlying: {spot_symbol}")
print(f" - UP breakout => BUY CE: {trade_ce}")
print(f" - DOWN breakout => BUY PE: {trade_pe}")
print(f" - Buffer pct: {BREAKOUT_BUFFER_PCT*100:.3f}%")
print(f" - PAPER_TRADE={PAPER_TRADE}")
print("-----------------------------------------------------")

# ----------------------------
# Funds + risk limits
# ----------------------------
initial_cash = kite_available_cash_once() if not PAPER_TRADE else 0.0
print(f"Broker Live Balance (once) = ₹{initial_cash:.2f} (ignored in paper mode)")

user_input = input("Enter fund to use for the day (e.g. 5000). Press Enter to use broker balance: ").strip()
fund = float(initial_cash) if user_input == "" else float(user_input)

usable_fund = fund * (1 - buffer_pct)
max_daily_loss = fund * MAX_DAILY_LOSS_PCT

write_log(f"Fund={fund}, usable={usable_fund}, max_daily_loss={max_daily_loss}, max_trades={MAX_TRADES_PER_DAY}")

trades_taken = 0
realized_pnl = 0.0
active_trade = None
rearm_ok = True  # for REARM_REQUIRED

# ----------------------------
# Spot samples storage
# ----------------------------
spot_samples = deque(maxlen=200000)

def get_prev_high_low(exclude_minutes=3):
    if not spot_samples:
        return None, None
    cutoff = now_ist() - timedelta(minutes=exclude_minutes)
    vals = [p for (ts, p) in spot_samples if ts <= cutoff]
    if len(vals) < 5:
        return None, None
    return max(vals), min(vals)

# ----------------------------
# Trade functions
# ----------------------------
def open_trade(direction, sym_to_trade):
    global active_trade, trades_taken, rearm_ok

    opt_ltp = get_ltp_any(key_for(sym_to_trade))
    if not opt_ltp:
        write_log(f"open_trade failed: no LTP for {sym_to_trade}")
        return False

    lot = infer_lot_size(sym_to_trade)
    max_u = max_units_per_order(lot)

    remaining = usable_fund
    buy_oids = []
    requested = []  # track chunks for paper
    est_ltp = float(opt_ltp)

    # buy chunks
    while True:
        chunk = aligned_affordable_chunk(remaining, est_ltp, lot, max_u, buffer_pct)
        if chunk <= 0:
            break

        if PAPER_TRADE:
            oid, _ = paper.buy_market(sym_to_trade, chunk, est_ltp)
            buy_oids.append(oid)
            requested.append({"oid": oid, "qty": int(chunk), "px": float(est_ltp)})
        else:
            oid = place_market_buy(sym_to_trade, chunk)
            buy_oids.append(oid)

        remaining = max(0.0, remaining - (est_ltp * int(chunk)))
        time.sleep(0.08)
        # refresh ltp
        est_ltp = get_ltp_any(key_for(sym_to_trade)) or est_ltp

    if not buy_oids:
        write_log(f"open_trade: no buys possible for {sym_to_trade}")
        return False

    # compute avg buy
    if PAPER_TRADE:
        avg_buy = float(paper.positions[sym_to_trade]["avg"])
        qty = int(paper.positions[sym_to_trade]["qty"])
    else:
        status_map = wait_for_orders(buy_oids, timeout_sec=15, poll_interval=0.35)
        qty = 0
        w_sum = 0.0
        for oid in buy_oids:
            od = status_map.get(oid, {}) or {}
            if od.get("status") == "REJECTED":
                continue
            fqty = int(od.get("filled_quantity") or od.get("quantity") or 0)
            apx  = float(od.get("average_price") or 0.0)
            if fqty > 0 and apx > 0:
                qty += fqty
                w_sum += apx * fqty
        if qty <= 0:
            # fallback
            ltp_now = get_ltp_any(key_for(sym_to_trade)) or est_ltp
            # assume last chunk qty minimal; but safer to abort
            write_log("open_trade: qty computed as 0 from broker; aborting.")
            return False
        avg_buy = w_sum / qty

    initial_sl = round(avg_buy * (1 - stoploss_percent), 1)

    sl_order_ids = []
    sl_entries = []
    if PAPER_TRADE:
        paper.set_sl(sym_to_trade, initial_sl)
    else:
        sl_entries = place_split_sl_orders(sym_to_trade, qty, initial_sl, lot)
        sl_order_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]

    active_trade = {
        "direction": direction,
        "sym": sym_to_trade,
        "qty": int(qty),
        "lot": int(lot),
        "avg_buy": float(avg_buy),
        "current_sl": float(initial_sl),
        "sl_entries": sl_entries,
        "sl_order_ids": sl_order_ids,
        "opened_at": now_ist(),
    }

    trades_taken += 1
    rearm_ok = False  # require rearm if enabled
    write_log(f"TRADE_OPEN {direction} sym={sym_to_trade} qty={qty} avg={avg_buy} SL={initial_sl} trades={trades_taken}")
    return True

def close_trade(reason, ltp):
    global active_trade, realized_pnl, trade_history

    sym = active_trade["sym"]
    qty = active_trade["qty"]
    avg_buy = active_trade["avg_buy"]
    direction = active_trade["direction"]

    buy_time = active_trade.get("opened_at")  # already present in your active_trade
    exit_time = now_ist()

    # Duration in seconds (safe)
    if buy_time:
        duration_sec = int((exit_time - buy_time).total_seconds())
    else:
        duration_sec = None



    if PAPER_TRADE:
        pnl = paper.exit_market(sym, ltp, reason=reason)
    else:
        safe_exit_real(
            sym,
            qty,
            active_trade.get("sl_order_ids", []),
            active_trade["lot"],
            reason=reason
        )
        pnl = (ltp - avg_buy) * qty  # best-effort

    realized_pnl += pnl

    trade_history.append({
        "symbol": sym,
        "direction": direction,
        "qty": qty,
        "avg_buy": round(avg_buy, 2),
        "exit_price": round(ltp, 2),
        "pnl": round(pnl, 2),
        "reason": reason,
        "buy_time": buy_time,
        "exit_time": exit_time,
        "duration_sec": duration_sec,
    })

    write_log(
        f"TRADE_CLOSE {('PAPER' if PAPER_TRADE else 'REAL')} "
        f"{reason} sym={sym} pnl={pnl:.2f} realized={realized_pnl:.2f}"
    )

    active_trade = None

def monitor_active_trade():
    global active_trade, realized_pnl
    if not active_trade:
        return False

    sym = active_trade["sym"]
    avg_buy = active_trade["avg_buy"]
    current_sl = active_trade["current_sl"]

    ltp = get_ltp_any(key_for(sym))
    if not ltp:
        return True

    profit_pct = (ltp - avg_buy) / avg_buy if avg_buy else 0.0

    # SL hit
    if ltp <= current_sl:
        close_trade("SL", ltp)
        return False

    # Trailing SL
    if profit_pct >= trail_start_percent:
        trail_step_value = avg_buy * trail_step_percent
        new_sl = round(ltp - trail_step_value, 1)
        if new_sl > current_sl:
            if PAPER_TRADE:
                paper.modify_sl(sym, new_sl)
            else:
                ok = 0
                for e in (active_trade.get("sl_entries") or []):
                    oid = e.get("order_id")
                    if oid and modify_sl_order(oid, new_sl):
                        ok += 1
                        e["price"] = float(new_sl)
                        time.sleep(0.08)
                write_log(f"TRAIL_SL REAL sym={sym} new_sl={new_sl} modified={ok}")
            active_trade["current_sl"] = float(new_sl)
            write_log(f"TRAIL_SL sym={sym} -> {new_sl}")

    # Target square-off
    if enable_square_off and profit_pct >= square_off_target_percent:
        close_trade("TARGET", ltp)
        return False

    return True

# ----------------------------
# Emergency stop
# ----------------------------
def emergency_exit(signum=None, frame=None):
    print("\nEMERGENCY STOP received.")
    write_log("EMERGENCY STOP received.")
    if active_trade:
        sym = active_trade["sym"]
        ltp = get_ltp_any(key_for(sym)) or active_trade["avg_buy"]
        close_trade("EMERGENCY", ltp)
    write_log("=== Breakout session ended by emergency ===")
    sys.exit(0)

signal.signal(signal.SIGINT, emergency_exit)

# ----------------------------
# Main loop
# ----------------------------
def in_trading_window(t: datetime):
    return START_TIME <= t.time() <= END_TIME

def past_force_exit(t: datetime):
    return t.time() >= FORCE_EXIT_TIME

print("Starting breakout loop... (Ctrl+C for emergency stop)")
write_log("Main loop started.")

# wait until start
while now_ist().time() < START_TIME:
    clear_output(wait=True)
    print("Waiting for start time 09:50 IST...")
    time.sleep(5)

last_display = 0

while True:
    t = now_ist()

    # end of day stop
    if t.time() > END_TIME:
        write_log("End time reached. Stopping.")
        break

    # spot sample
    spot = get_ltp_any(spot_symbol)
    if spot:
        spot_samples.append((t, float(spot)))

    prev_high, prev_low = get_prev_high_low(EXCLUDE_LAST_MINUTES)
    if prev_high is not None and prev_low is not None and spot is not None:
        up_level = prev_high * (1 + BREAKOUT_BUFFER_PCT)
        dn_level = prev_low  * (1 - BREAKOUT_BUFFER_PCT)
    else:
        up_level = dn_level = None

    # If trade open: monitor
    if active_trade:
        # max loss check includes unrealized
        ltp = get_ltp_any(key_for(active_trade["sym"])) or active_trade["avg_buy"]
        unreal = (ltp - active_trade["avg_buy"]) * active_trade["qty"]
        if (realized_pnl + unreal) <= -max_daily_loss:
            write_log("Max daily loss hit (realized+unrealized). Forcing exit & stop.")
            close_trade("MAX_LOSS", ltp)
            break

        # force exit near close
        if past_force_exit(t):
            close_trade("TIME_EXIT", ltp)
            break

        monitor_active_trade()
        time.sleep(MONITOR_SLEEP_SEC)
        continue

    # If no trade open: risk and trade count
    if realized_pnl <= -max_daily_loss:
        write_log("Max daily loss hit (realized). Stopping.")
        break

    if trades_taken >= MAX_TRADES_PER_DAY:
        if time.time() - last_display > 5:
            clear_output(wait=True)
            print(f"[{t.time().strftime('%H:%M:%S')}] Max trades reached ({trades_taken}). No new entries.")
            print(f"RealizedPnL={realized_pnl:.2f}")
            last_display = time.time()
        time.sleep(SPOT_POLL_SEC)
        continue

    if not in_trading_window(t) or up_level is None or dn_level is None or spot is None:
        time.sleep(SPOT_POLL_SEC)
        continue

    # Rearm logic: wait until spot is "inside" band before new breakout trade
    if REARM_REQUIRED and not rearm_ok:
        if dn_level < spot < up_level:
            rearm_ok = True
            write_log("Re-armed for next trade (spot back inside band).")
        else:
            time.sleep(SPOT_POLL_SEC)
            continue

    direction = None
    if spot > up_level:
        direction = "UP"
    elif spot < dn_level:
        direction = "DOWN"

    # display
    if time.time() - last_display > 2.5:
        clear_output(wait=True)
        print("=== Breakout Scanner (Cell4) ===")
        print(f"Time: {t.time().strftime('%H:%M:%S')} IST")
        print(f"Spot: {spot:.2f}")
        print(f"PrevHigh(excl {EXCLUDE_LAST_MINUTES}m): {prev_high:.2f} | UpTrig: {up_level:.2f}")
        print(f"PrevLow (excl {EXCLUDE_LAST_MINUTES}m): {prev_low:.2f} | DnTrig: {dn_level:.2f}")
        print(f"Trades: {trades_taken}/{MAX_TRADES_PER_DAY} | RealizedPnL: {realized_pnl:.2f} | MaxLoss: {-max_daily_loss:.2f}")
        print(f"PAPER_TRADE={PAPER_TRADE} | REARM_REQUIRED={REARM_REQUIRED}")
        last_display = time.time()

    # Entry
    if direction == "UP":
        write_log(f"BREAKOUT UP spot={spot:.2f} > {up_level:.2f}. Enter CE={trade_ce}")
        if open_trade("UP", trade_ce):
            print(f"Entered UP trade: BUY {trade_ce}")
    elif direction == "DOWN":
        write_log(f"BREAKOUT DOWN spot={spot:.2f} < {dn_level:.2f}. Enter PE={trade_pe}")
        if open_trade("DOWN", trade_pe):
            print(f"Entered DOWN trade: BUY {trade_pe}")

    time.sleep(SPOT_POLL_SEC)

write_log("=== Breakout session ended normally ===")
print("Breakout session ended.")

def format_hms(seconds):
    if seconds is None:
        return "NA"
    seconds = int(seconds)
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    if h > 0:
        return f"{h}h {m}m {s}s"
    if m > 0:
        return f"{m}m {s}s"
    return f"{s}s"

def print_trade_summary():
    print("\n" + "="*60)
    print("📊 DAILY TRADE SUMMARY")
    print("="*60)

    total_trades = len(trade_history)
    wins = sum(1 for t in trade_history if t["pnl"] > 0)
    losses = sum(1 for t in trade_history if t["pnl"] < 0)

    pnl_pct = (realized_pnl / fund * 100) if fund > 0 else 0.0

    print(f"Mode               : {'PAPER' if PAPER_TRADE else 'LIVE'}")
    print(f"Fund Used          : ₹{fund:.2f}")
    print(f"Max Daily Loss     : ₹{max_daily_loss:.2f} ({MAX_DAILY_LOSS_PCT*100:.1f}%)")
    print("-"*60)
    print(f"Total Trades       : {total_trades}")
    print(f"Winning Trades     : {wins}")
    print(f"Losing Trades      : {losses}")
    print("-"*60)
    print(f"Net PnL            : ₹{realized_pnl:.2f}")
    print(f"Net PnL %          : {pnl_pct:.2f}%")
    print("="*60)

    if trade_history:
        print("\n🧾 Trade-wise details:")
        for i, t in enumerate(trade_history, 1):
            bt = t.get("buy_time")
            et = t.get("exit_time")
            dur_str = format_hms(t.get("duration_sec"))

            # format times safely
            bt_str = bt.strftime("%H:%M:%S") if bt else "NA"
            et_str = et.strftime("%H:%M:%S") if et else "NA"

            print(
                f"{i}. {t['symbol']} | {t['direction']} | "
                f"Qty:{t['qty']} | "
                f"Buy:{t['avg_buy']} → Exit:{t['exit_price']} | "
                f"PnL:{t['pnl']} | {t['reason']} | "
                f"BuyTime:{bt_str} | ExitTime:{et_str} | Duration:{dur_str}"
            )
    else:
        print("No trades executed today.")
print_trade_summary()

In [ ]:
# @title
# ============================================
# CELL 4 — Hybrid Breakout Auto Trader (09:50–15:30)
# Hybrid levels: Before 11:00 => ORB (09:15–09:45), After 11:00 => Rolling range
# Entry confirmation: 1-minute candle CLOSE beyond trigger + buffer
# History: seeded from market open so logic does not depend on runtime start
# Includes: paper trade option, daily loss cap, max trades/day, summary (times + duration)
# ============================================

import os, time, sys, signal
from datetime import datetime, date, timedelta, time as dtime
from zoneinfo import ZoneInfo
from collections import deque
from IPython.display import clear_output

# ----------------------------
# REQUIRED VARS (expected like Cell3)
# ----------------------------
required_vars = [
    "stoploss_percent",
    "trail_start_percent",
    "trail_step_percent",
    "square_off_target_percent",
    "log_folder",
]
missing = [v for v in required_vars if v not in globals() or globals()[v] is None]
if missing:
    raise NameError(f"Missing required vars: {missing}. Define before running Cell4.")

# ----------------------------
# CONFIG
# ----------------------------
IST = ZoneInfo("Asia/Kolkata")

# Strategy run window (scanner)
START_TIME = dtime(9, 50)
END_TIME   = dtime(15, 30)
FORCE_EXIT_TIME = dtime(15, 29, 50)

# Market open
MARKET_OPEN_TIME = dtime(9, 15)

# Hybrid regime
REGIME_SWITCH_TIME = dtime(11, 0)     # before -> ORB, after -> rolling
ORB_START_TIME = dtime(9, 15)
ORB_END_TIME   = dtime(9, 45)         # ✅ as requested

# Rolling window after 11:00
ROLLING_MINUTES = 45

# Exclude last X minutes while building levels
EXCLUDE_LAST_MINUTES = 3

# Breakout buffer in percentage (e.g., 0.0015 = 0.15%)
BREAKOUT_BUFFER_PCT = 0.0002

# Risk & limits
MAX_TRADES_PER_DAY = 3
MAX_DAILY_LOSS_PCT = 0.20

# Paper trade option
PAPER_TRADE = False   # set False for LIVE

# Polling
SPOT_POLL_SEC = 1.0
MONITOR_SLEEP_SEC = 1.5

# Rearm: after a trade closes, require price to return inside band before next entry
REARM_REQUIRED = True

# Fund usage buffer
buffer_pct = globals().get("buffer_pct", 0.02)

# Exchange prefix from Cell2
exchange_prefix = globals().get("current_prefix", "NFO")

# Enable square-off target logic
enable_square_off = globals().get("enable_square_off", True)

# Freeze limit per order
DEFAULT_MAX_FREEZE_QTY = globals().get(
    "max_freeze_qty_per_order",
    1800 if exchange_prefix == "NFO" else 600
)

# ----------------------------
# Logging
# ----------------------------
os.makedirs(log_folder, exist_ok=True)
log_file_path = os.path.join(log_folder, date.today().isoformat() + "_hybrid_breakout.txt")

def write_log(line: str):
    ts = datetime.now(IST).isoformat(timespec="seconds")
    with open(log_file_path, "a") as f:
        f.write(f"{ts} | {line}\n")

write_log("=== Hybrid Breakout session started (Cell4) ===")

def now_ist():
    return datetime.now(IST)

def combine_today(t: dtime):
    return datetime.combine(date.today(), t, tzinfo=IST)

def key_for(sym: str) -> str:
    return f"{exchange_prefix}:{sym}"

# ----------------------------
# Kite helpers
# ----------------------------
def kite_available_cash_once():
    try:
        m = kite.margins()
        return float(m["equity"]["available"]["live_balance"])
    except Exception as e:
        write_log(f"Could not fetch kite margins: {e}")
        return 0.0

def get_ltp_any(full_symbol: str):
    """full_symbol like 'NSE:NIFTY 50' or 'NFO:XYZ'"""
    try:
        d = kite.ltp([full_symbol])
        return d.get(full_symbol, {}).get("last_price")
    except Exception as e:
        write_log(f"LTP fetch failed {full_symbol}: {e}")
        return None

def get_ltps(symbols):
    if not symbols:
        return {}
    keys = [key_for(s) for s in symbols]
    try:
        data = kite.ltp(keys)
        return {s: data.get(key_for(s), {}).get("last_price") for s in symbols}
    except Exception as e:
        write_log(f"get_ltps failed: {e}")
        return {s: None for s in symbols}

TERMINAL = {"COMPLETE", "REJECTED", "CANCELLED"}

def fetch_orders_map():
    try:
        all_orders = kite.orders()
        return {o.get("order_id"): o for o in all_orders if o.get("order_id")}
    except Exception as e:
        write_log(f"fetch_orders_map failed: {e}")
        return {}

def wait_for_orders(order_ids, timeout_sec=15, poll_interval=0.35):
    order_ids = [oid for oid in order_ids if oid]
    if not order_ids:
        return {}
    start = time.time()
    last_seen = {}
    while time.time() - start < timeout_sec:
        omap = fetch_orders_map()
        for oid in order_ids:
            if oid in omap:
                last_seen[oid] = omap[oid]
        done = sum(1 for oid in order_ids if (last_seen.get(oid, {}) or {}).get("status") in TERMINAL)
        if done == len(order_ids):
            return last_seen
        time.sleep(poll_interval)
    write_log(f"wait_for_orders timeout after {timeout_sec}s; returning partial.")
    return last_seen

# ----------------------------
# LIVE order wrappers
# ----------------------------
def place_market_buy(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_BUY,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_market_sell(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_sl_sell(sym, qty, sl_price):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_SL,
        price=float(sl_price),
        trigger_price=float(sl_price)
    )

def cancel_order_fast(order_id):
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)
        return True
    except Exception as e:
        write_log(f"cancel_order_fast ignore {order_id}: {e}")
        return False

def modify_sl_order(order_id, new_price):
    try:
        kite.modify_order(
            variety=kite.VARIETY_REGULAR,
            order_id=order_id,
            price=float(new_price),
            trigger_price=float(new_price)
        )
        return True
    except Exception as e:
        write_log(f"modify_sl_order failed {order_id} -> {new_price}: {e}")
        return False

# ----------------------------
# Freeze/lot utilities
# ----------------------------
def infer_lot_size(sym: str) -> int:
    if "SENSEX" in sym.upper() or exchange_prefix == "BFO":
        return 20
    return 65

def max_units_per_order(lot_size: int) -> int:
    max_units = int(DEFAULT_MAX_FREEZE_QTY)
    lots = max(1, max_units // lot_size)
    return int(lots * lot_size)

def aligned_affordable_chunk(remaining_funds, ltp, lot_size, max_units, buffer_pct):
    if not ltp or ltp <= 0:
        return 0
    affordable_units = int(remaining_funds // (ltp * (1 + buffer_pct)))
    affordable_lots = affordable_units // lot_size
    if affordable_lots <= 0:
        return 0
    units = affordable_lots * lot_size
    return int(min(units, max_units))

def split_units_to_chunks(total_units, max_units, lot_size):
    chunks = []
    remaining = int(total_units)
    if max_units < lot_size:
        max_units = lot_size
    while remaining > 0:
        take = min(remaining, max_units)
        rem_mod = take % lot_size
        if rem_mod != 0:
            take -= rem_mod
        if take <= 0:
            take = lot_size
        chunks.append(int(take))
        remaining -= take
    return chunks

def place_split_sl_orders(sym, total_units, sl_price, lot_size):
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(total_units, max_u, lot_size)
    placed = []
    for chunk in chunks:
        try:
            oid = place_sl_sell(sym, chunk, sl_price)
            placed.append({"order_id": oid, "qty": int(chunk), "price": float(sl_price)})
            time.sleep(0.08)
        except Exception as e:
            write_log(f"ERROR placing SL chunk {sym} qty={chunk}: {e}")
    return placed

def safe_exit_real(sym, qty, sl_order_ids, lot_size, reason="exit"):
    write_log(f"[safe_exit_real] {sym} qty={qty} reason={reason}")
    # cancel SLs
    for oid in (sl_order_ids or []):
        if oid:
            cancel_order_fast(oid)
            time.sleep(0.08)
    # sell chunks
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(qty, max_u, lot_size)
    for chunk in chunks:
        try:
            place_market_sell(sym, chunk)
            time.sleep(0.08)
        except Exception as e:
            write_log(f"ERROR market sell {sym} chunk={chunk}: {e}")

# ----------------------------
# Paper broker simulator
# ----------------------------
class PaperBroker:
    def __init__(self):
        self._oid = 100000
        self.positions = {}  # sym -> {qty, w_sum, avg, sl, open}
    def _next_oid(self):
        self._oid += 1
        return str(self._oid)
    def buy_market(self, sym, qty, ltp):
        oid = self._next_oid()
        pos = self.positions.get(sym, {"qty": 0, "w_sum": 0.0, "open": False})
        pos["qty"] += int(qty)
        pos["w_sum"] += float(ltp) * int(qty)
        pos["avg"] = pos["w_sum"] / pos["qty"]
        pos["open"] = True
        self.positions[sym] = pos
        return oid, pos["avg"]
    def set_sl(self, sym, sl_price):
        if sym in self.positions:
            self.positions[sym]["sl"] = float(sl_price)
    def modify_sl(self, sym, new_sl):
        if sym in self.positions:
            self.positions[sym]["sl"] = float(new_sl)
    def exit_market(self, sym, ltp, reason="exit"):
        if sym not in self.positions or not self.positions[sym].get("open"):
            return 0.0
        pos = self.positions[sym]
        qty = int(pos["qty"])
        avg = float(pos["avg"])
        pnl = (float(ltp) - avg) * qty
        pos["open"] = False
        pos["exit_reason"] = reason
        pos["realized_pnl"] = pnl
        return pnl

paper = PaperBroker()

# ----------------------------
# Selection (from Cell2)
# ----------------------------
selected = get_selected_symbols()
if not selected:
    raise SystemExit("No symbols selected. Select in chain UI (Cell2) and re-run Cell4.")

if "symbol_meta" not in globals() or not symbol_meta:
    raise SystemExit("symbol_meta missing. Re-render selector chain (Cell2) and re-select.")

selected_ce = [s for s in selected if (symbol_meta.get(s, {}) or {}).get("instrument_type") == "CE"]
selected_pe = [s for s in selected if (symbol_meta.get(s, {}) or {}).get("instrument_type") == "PE"]
if not selected_ce or not selected_pe:
    raise SystemExit("Need at least ONE CE and ONE PE selected for UP/DOWN breakouts.")

first_meta = symbol_meta[selected[0]]
index_name = first_meta.get("index_name")

# underlying spot symbol from INDEX_CONFIG (Cell2)
spot_symbol = None
if "INDEX_CONFIG" in globals():
    for _, cfg in INDEX_CONFIG.items():
        if cfg.get("name") == index_name:
            spot_symbol = cfg.get("spot")
            break
if not spot_symbol:
    raise SystemExit("Could not determine underlying spot symbol from INDEX_CONFIG. Re-run Cell2.")

atm_data = (atm_info.get(index_name, {}) if "atm_info" in globals() else {}) or {}
atm_strike = atm_data.get("atm_strike")

def strike_distance(sym):
    if atm_strike is None:
        return 10**18
    return abs((symbol_meta.get(sym, {}) or {}).get("strike", 10**18) - atm_strike)

# nearest selected CE/PE
trade_ce = sorted(selected_ce, key=strike_distance)[0]
trade_pe = sorted(selected_pe, key=strike_distance)[0]

print("Hybrid Breakout strategy configured:")
print(f" - Underlying: {spot_symbol}")
print(f" - ORB: {ORB_START_TIME.strftime('%H:%M')}–{ORB_END_TIME.strftime('%H:%M')} (before {REGIME_SWITCH_TIME.strftime('%H:%M')})")
print(f" - After {REGIME_SWITCH_TIME.strftime('%H:%M')}: Rolling {ROLLING_MINUTES} minutes")
print(f" - UP breakout => BUY CE: {trade_ce}")
print(f" - DOWN breakout => BUY PE: {trade_pe}")
print(f" - Buffer: {BREAKOUT_BUFFER_PCT*100:.3f}% | Exclude last {EXCLUDE_LAST_MINUTES}m")
print(f" - PAPER_TRADE={PAPER_TRADE}")
print("-----------------------------------------------------")

write_log(f"Configured spot={spot_symbol}, CE={trade_ce}, PE={trade_pe}, ORB=09:15-09:45, switch=11:00, rolling={ROLLING_MINUTES}m")

# ----------------------------
# Funds + risk
# ----------------------------
initial_cash = kite_available_cash_once() if not PAPER_TRADE else 0.0
print(f"Broker Live Balance (once) = ₹{initial_cash:.2f} (ignored in paper mode)")

user_input = input("Enter fund to use for the day (e.g. 5000). Press Enter to use broker balance: ").strip()
fund = float(initial_cash) if user_input == "" else float(user_input)

usable_fund = fund * (1 - buffer_pct)
max_daily_loss = fund * MAX_DAILY_LOSS_PCT

write_log(f"Fund={fund}, usable={usable_fund}, max_daily_loss={max_daily_loss}, max_trades={MAX_TRADES_PER_DAY}")

# ----------------------------
# Trade tracking + summary
# ----------------------------
trades_taken = 0
realized_pnl = 0.0
active_trade = None
rearm_ok = True

trade_history = []  # stores trade-wise details incl times + duration

def format_hms(seconds):
    if seconds is None:
        return "NA"
    seconds = int(seconds)
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    if h > 0:
        return f"{h}h {m}m {s}s"
    if m > 0:
        return f"{m}m {s}s"
    return f"{s}s"

def print_trade_summary():
    print("\n" + "="*70)
    print("📊 DAILY TRADE SUMMARY")
    print("="*70)

    total_trades = len(trade_history)
    wins = sum(1 for t in trade_history if t["pnl"] > 0)
    losses = sum(1 for t in trade_history if t["pnl"] < 0)
    pnl_pct = (realized_pnl / fund * 100) if fund > 0 else 0.0

    print(f"Mode               : {'PAPER' if PAPER_TRADE else 'LIVE'}")
    print(f"Fund Used          : ₹{fund:.2f}")
    print(f"Max Daily Loss     : ₹{max_daily_loss:.2f} ({MAX_DAILY_LOSS_PCT*100:.1f}%)")
    print("-"*70)
    print(f"Total Trades       : {total_trades}")
    print(f"Winning Trades     : {wins}")
    print(f"Losing Trades      : {losses}")
    print("-"*70)
    print(f"Net PnL            : ₹{realized_pnl:.2f}")
    print(f"Net PnL %          : {pnl_pct:.2f}%")
    print("="*70)

    if trade_history:
        print("\n🧾 Trade-wise details:")
        for i, t in enumerate(trade_history, 1):
            bt = t.get("buy_time")
            et = t.get("exit_time")
            bt_str = bt.strftime("%H:%M:%S") if bt else "NA"
            et_str = et.strftime("%H:%M:%S") if et else "NA"
            dur_str = format_hms(t.get("duration_sec"))
            print(
                f"{i}. {t['symbol']} | {t['direction']} | Qty:{t['qty']} | "
                f"Buy:{t['avg_buy']} → Exit:{t['exit_price']} | "
                f"PnL:{t['pnl']} | {t['reason']} | "
                f"BuyTime:{bt_str} | ExitTime:{et_str} | Duration:{dur_str}"
            )
    else:
        print("No trades executed today.")

# ----------------------------
# 1-minute candle store (seed + live build)
# ----------------------------
minute_candles = deque(maxlen=20000)

_current_minute = None
_curr_candle = None

def floor_to_minute(dt: datetime):
    return dt.replace(second=0, microsecond=0)

def find_instrument_token_for_spot(spot_symbol: str):
    """
    spot_symbol like 'NSE:NIFTY 50' or 'BSE:SENSEX'
    return instrument_token if found
    """
    try:
        ex, ts = spot_symbol.split(":")
        inst = kite.instruments(ex)
        for i in inst:
            if i.get("tradingsymbol") == ts:
                return i.get("instrument_token")
    except Exception as e:
        write_log(f"Token lookup failed for {spot_symbol}: {e}")
    return None

def seed_minute_candles_from_history(start_dt: datetime, end_dt: datetime):
    token = find_instrument_token_for_spot(spot_symbol)
    if not token:
        write_log("Could not seed history: instrument_token not found. Live candles only.")
        return
    try:
        candles = kite.historical_data(token, start_dt, end_dt, "minute", continuous=False, oi=False)
        added = 0
        for c in candles:
            ts = c.get("date")
            if not ts:
                continue
            start_ts = ts.astimezone(IST)
            end_ts = start_ts + timedelta(minutes=1)
            minute_candles.append({
                "end": end_ts,
                "open": float(c["open"]),
                "high": float(c["high"]),
                "low": float(c["low"]),
                "close": float(c["close"]),
            })
            added += 1
        write_log(f"Seeded {added} minute candles from {start_dt.time()} to {end_dt.time()}.")
    except Exception as e:
        write_log(f"Seeding minute candles failed: {e}")

def update_live_minute_candle(ltp: float, ts: datetime):
    global _current_minute, _curr_candle
    if ltp is None:
        return
    m = floor_to_minute(ts)

    if _current_minute is None:
        _current_minute = m
        _curr_candle = {"end": m + timedelta(minutes=1),
                        "open": float(ltp), "high": float(ltp),
                        "low": float(ltp), "close": float(ltp)}
        return

    if m != _current_minute:
        if _curr_candle is not None:
            minute_candles.append(_curr_candle)
        _current_minute = m
        _curr_candle = {"end": m + timedelta(minutes=1),
                        "open": float(ltp), "high": float(ltp),
                        "low": float(ltp), "close": float(ltp)}
        return

    _curr_candle["high"] = max(_curr_candle["high"], float(ltp))
    _curr_candle["low"]  = min(_curr_candle["low"], float(ltp))
    _curr_candle["close"] = float(ltp)

def last_closed_candle(now_dt: datetime):
    if not minute_candles:
        return None
    cur_min = floor_to_minute(now_dt)
    for c in reversed(minute_candles):
        if c["end"] <= cur_min:
            return c
    return None

# Seed history from market open -> now (so runtime start doesn't matter)
seed_start = combine_today(MARKET_OPEN_TIME)
seed_end = now_ist()
if seed_end > seed_start:
    seed_minute_candles_from_history(seed_start, seed_end)

# Initialize live candle builder with current spot if possible
spot_now = get_ltp_any(spot_symbol)
if spot_now is not None:
    update_live_minute_candle(float(spot_now), now_ist())

# ----------------------------
# Hybrid level computation (with times)
# ----------------------------
def compute_levels_with_times(now_dt: datetime):
    """
    Returns:
      regime, hi, hi_time, lo, lo_time, window_start, window_end
    Times are candle-end times of the candles where high/low occurred.
    """
    if not minute_candles:
        return None, None, None, None, None, None, None

    cutoff = now_dt - timedelta(minutes=EXCLUDE_LAST_MINUTES)

    if now_dt.time() < REGIME_SWITCH_TIME:
        regime = "ORB"
        w_start = combine_today(ORB_START_TIME)
        w_end_fixed = combine_today(ORB_END_TIME)
        window_end = min(w_end_fixed, cutoff)
        window_start = w_start
    else:
        regime = "ROLL"
        window_end = cutoff
        window_start = window_end - timedelta(minutes=ROLLING_MINUTES)

    relevant = [c for c in minute_candles if (c["end"] > window_start and c["end"] <= window_end)]
    if len(relevant) < 5:
        return regime, None, None, None, None, window_start, window_end

    hi_candle = max(relevant, key=lambda x: x["high"])
    lo_candle = min(relevant, key=lambda x: x["low"])

    return (
        regime,
        hi_candle["high"], hi_candle["end"],
        lo_candle["low"],  lo_candle["end"],
        window_start, window_end
    )

# ----------------------------
# Trade open/close/monitor
# ----------------------------
def open_trade(direction, sym_to_trade):
    global active_trade, trades_taken, rearm_ok

    opt_ltp = get_ltp_any(key_for(sym_to_trade))
    if not opt_ltp:
        write_log(f"open_trade failed: no LTP for {sym_to_trade}")
        return False

    lot = infer_lot_size(sym_to_trade)
    max_u = max_units_per_order(lot)

    remaining = usable_fund
    buy_oids = []
    est_ltp = float(opt_ltp)

    # buy in chunks
    while True:
        chunk = aligned_affordable_chunk(remaining, est_ltp, lot, max_u, buffer_pct)
        if chunk <= 0:
            break

        if PAPER_TRADE:
            oid, _ = paper.buy_market(sym_to_trade, chunk, est_ltp)
            buy_oids.append(oid)
        else:
            oid = place_market_buy(sym_to_trade, chunk)
            buy_oids.append(oid)

        remaining = max(0.0, remaining - (est_ltp * int(chunk)))
        time.sleep(0.08)
        est_ltp = get_ltp_any(key_for(sym_to_trade)) or est_ltp

    if not buy_oids:
        write_log(f"open_trade: no buys possible for {sym_to_trade}")
        return False

    # compute avg buy
    if PAPER_TRADE:
        avg_buy = float(paper.positions[sym_to_trade]["avg"])
        qty = int(paper.positions[sym_to_trade]["qty"])
    else:
        status_map = wait_for_orders(buy_oids, timeout_sec=15, poll_interval=0.35)
        qty = 0
        w_sum = 0.0
        for oid in buy_oids:
            od = status_map.get(oid, {}) or {}
            if od.get("status") == "REJECTED":
                continue
            fqty = int(od.get("filled_quantity") or od.get("quantity") or 0)
            apx  = float(od.get("average_price") or 0.0)
            if fqty > 0 and apx > 0:
                qty += fqty
                w_sum += apx * fqty
        if qty <= 0:
            write_log("open_trade: qty computed as 0 from broker; aborting.")
            return False
        avg_buy = w_sum / qty

    initial_sl = round(avg_buy * (1 - stoploss_percent), 1)

    sl_entries = []
    sl_ids = []
    if PAPER_TRADE:
        paper.set_sl(sym_to_trade, initial_sl)
    else:
        sl_entries = place_split_sl_orders(sym_to_trade, qty, initial_sl, lot)
        sl_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]

    active_trade = {
        "direction": direction,
        "sym": sym_to_trade,
        "qty": int(qty),
        "lot": int(lot),
        "avg_buy": float(avg_buy),
        "current_sl": float(initial_sl),
        "sl_entries": sl_entries,
        "sl_order_ids": sl_ids,
        "opened_at": now_ist(),   # buy time
    }

    trades_taken += 1
    rearm_ok = False

    write_log(f"TRADE_OPEN {direction} sym={sym_to_trade} qty={qty} avg={avg_buy:.2f} SL={initial_sl} trades={trades_taken}")
    return True

def close_trade(reason, ltp):
    global active_trade, realized_pnl, trade_history

    sym = active_trade["sym"]
    qty = active_trade["qty"]
    avg_buy = active_trade["avg_buy"]
    direction = active_trade["direction"]

    buy_time = active_trade.get("opened_at")
    exit_time = now_ist()
    duration_sec = int((exit_time - buy_time).total_seconds()) if buy_time else None

    if PAPER_TRADE:
        pnl = paper.exit_market(sym, ltp, reason=reason)
    else:
        safe_exit_real(sym, qty, active_trade.get("sl_order_ids", []), active_trade["lot"], reason=reason)
        pnl = (ltp - avg_buy) * qty  # best-effort estimate

    realized_pnl += pnl

    trade_history.append({
        "symbol": sym,
        "direction": direction,
        "qty": qty,
        "avg_buy": round(avg_buy, 2),
        "exit_price": round(float(ltp), 2),
        "pnl": round(float(pnl), 2),
        "reason": reason,
        "buy_time": buy_time,
        "exit_time": exit_time,
        "duration_sec": duration_sec,
    })

    write_log(f"TRADE_CLOSE {('PAPER' if PAPER_TRADE else 'REAL')} {reason} sym={sym} pnl={pnl:.2f} realized={realized_pnl:.2f}")
    active_trade = None

def monitor_active_trade():
    """
    Monitor SL/trailing/target more frequently than candle closes.
    Returns True if still open, False if closed.
    """
    global active_trade

    if not active_trade:
        return False

    sym = active_trade["sym"]
    avg_buy = active_trade["avg_buy"]
    current_sl = active_trade["current_sl"]

    ltp = get_ltp_any(key_for(sym))
    if not ltp:
        return True

    profit_pct = (ltp - avg_buy) / avg_buy if avg_buy else 0.0

    # SL hit
    if ltp <= current_sl:
        close_trade("SL", ltp)
        return False

    # Trailing SL
    if profit_pct >= trail_start_percent:
        trail_step_value = avg_buy * trail_step_percent
        new_sl = round(ltp - trail_step_value, 1)
        if new_sl > current_sl:
            if PAPER_TRADE:
                paper.modify_sl(sym, new_sl)
            else:
                ok = 0
                for e in (active_trade.get("sl_entries") or []):
                    oid = e.get("order_id")
                    if oid and modify_sl_order(oid, new_sl):
                        ok += 1
                        e["price"] = float(new_sl)
                        time.sleep(0.08)
                write_log(f"TRAIL_SL REAL sym={sym} new_sl={new_sl} modified={ok}")
            active_trade["current_sl"] = float(new_sl)
            write_log(f"TRAIL_SL sym={sym} -> {new_sl}")

    # Target square-off
    if enable_square_off and profit_pct >= square_off_target_percent:
        close_trade("TARGET", ltp)
        return False

    return True

# ----------------------------
# Emergency stop
# ----------------------------
def emergency_exit(signum=None, frame=None):
    print("\nEMERGENCY STOP received.")
    write_log("EMERGENCY STOP received.")
    if active_trade:
        sym = active_trade["sym"]
        ltp = get_ltp_any(key_for(sym)) or active_trade["avg_buy"]
        close_trade("EMERGENCY", ltp)
    write_log("=== Hybrid Breakout session ended by emergency ===")
    print_trade_summary()
    sys.exit(0)

signal.signal(signal.SIGINT, emergency_exit)

# ----------------------------
# Main loop: hybrid levels + 1m close confirmation
# ----------------------------
def in_trading_window(t: datetime):
    return START_TIME <= t.time() <= END_TIME

def past_force_exit(t: datetime):
    return t.time() >= FORCE_EXIT_TIME

print("Starting Hybrid Breakout loop... (Ctrl+C for emergency stop)")
write_log("Main loop started.")

# wait until START_TIME
while now_ist().time() < START_TIME:
    clear_output(wait=True)
    print("Waiting for start time 09:50 IST...")
    time.sleep(5)

last_signal_candle_end = None

while True:
    t = now_ist()

    # end of day
    if t.time() > END_TIME:
        write_log("End time reached. Stopping.")
        break

    # update live candle builder from spot LTP
    spot = get_ltp_any(spot_symbol)
    if spot is not None:
        update_live_minute_candle(float(spot), t)

    # If trade open: risk check + monitor
    if active_trade:
        # daily loss check includes unrealized
        ltp = get_ltp_any(key_for(active_trade["sym"])) or active_trade["avg_buy"]
        unreal = (ltp - active_trade["avg_buy"]) * active_trade["qty"]
        # --- NEW: compute unrealized pnl % for display (no behavior change) ---
        entry_value = active_trade["avg_buy"] * active_trade["qty"]
        unreal_pct = (unreal / entry_value * 100) if entry_value > 0 else 0.0

        # --- NEW: print unrealized pnl line (keeps existing output, adds one extra line) ---
        print(f"Open Trade: {active_trade['sym']} | Qty:{active_trade['qty']} | "
              f"Avg:{active_trade['avg_buy']:.2f} | LTP:{ltp:.2f} | "
              f"UnrealizedPnL: ₹{unreal:.2f} ({unreal_pct:.2f}%)")
        if (realized_pnl + unreal) <= -max_daily_loss:
            write_log("Max daily loss hit (realized+unrealized). Forcing exit & stop.")
            close_trade("MAX_LOSS", ltp)
            break

        # force exit near market close
        if past_force_exit(t):
            write_log("Force exit time reached. Closing open trade.")
            close_trade("TIME_EXIT", ltp)
            break

        monitor_active_trade()
        time.sleep(MONITOR_SLEEP_SEC)
        continue

    # no active trade: stop if max daily loss reached (realized)
    if realized_pnl <= -max_daily_loss:
        write_log("Max daily loss hit (realized). Stopping.")
        break

    # no new entries after max trades
    if trades_taken >= MAX_TRADES_PER_DAY:
        clear_output(wait=True)
        print(f"[{t.strftime('%H:%M:%S')}] Max trades reached ({trades_taken}). No new entries.")
        print(f"RealizedPnL={realized_pnl:.2f}")
        time.sleep(SPOT_POLL_SEC)
        continue

    # operate only inside trading window
    if not in_trading_window(t):
        time.sleep(SPOT_POLL_SEC)
        continue

    # 1-minute close confirmation: only process when we have a NEW closed candle
    lc = last_closed_candle(t)
    if lc is None:
        time.sleep(SPOT_POLL_SEC)
        continue

    if last_signal_candle_end == lc["end"]:
        time.sleep(SPOT_POLL_SEC)
        continue

    close_px = lc["close"]
    close_time = lc["end"]

    regime, prev_high, prev_high_time, prev_low, prev_low_time, w_start, w_end = compute_levels_with_times(t)

    # mark processed candle regardless (prevents spam)
    last_signal_candle_end = lc["end"]

    if prev_high is None or prev_low is None:
        time.sleep(SPOT_POLL_SEC)
        continue

    up_level = prev_high * (1 + BREAKOUT_BUFFER_PCT)
    dn_level = prev_low  * (1 - BREAKOUT_BUFFER_PCT)

    # Rearm check (use close confirmation band)
    if REARM_REQUIRED and not rearm_ok:
        if dn_level < close_px < up_level:
            rearm_ok = True
            write_log("Re-armed for next trade (close back inside band).")
        time.sleep(SPOT_POLL_SEC)
        continue

    # Print scanner state including high/low times
    clear_output(wait=True)
    print("=== Hybrid Breakout Scanner (1m Close Confirm) ===")
    print(f"Time now: {t.strftime('%H:%M:%S')} IST | Regime: {regime}")
    print(f"Level window: {w_start.strftime('%H:%M')} → {w_end.strftime('%H:%M')} (excl last {EXCLUDE_LAST_MINUTES}m)")
    print(f"PrevHigh: {prev_high:.2f} at {prev_high_time.strftime('%H:%M')} | UpTrig(+buf): {up_level:.2f}")
    print(f"PrevLow : {prev_low:.2f} at {prev_low_time.strftime('%H:%M')} | DnTrig(-buf): {dn_level:.2f}")
    print(f"Last closed 1m close: {close_px:.2f} at {close_time.strftime('%H:%M')}")
    print(f"Trades: {trades_taken}/{MAX_TRADES_PER_DAY} | RealizedPnL: {realized_pnl:.2f} | PAPER={PAPER_TRADE}")
    print("-"*60)

    # Decide entry ONLY on candle close confirmation
    direction = None
    if close_px > up_level:
        direction = "UP"
    elif close_px < dn_level:
        direction = "DOWN"

    if direction == "UP":
        write_log(f"BREAKOUT UP (1m close) close={close_px:.2f} > {up_level:.2f}. Enter CE={trade_ce}")
        if open_trade("UP", trade_ce):
            print(f"Entered UP trade: BUY {trade_ce}")
    elif direction == "DOWN":
        write_log(f"BREAKOUT DOWN (1m close) close={close_px:.2f} < {dn_level:.2f}. Enter PE={trade_pe}")
        if open_trade("DOWN", trade_pe):
            print(f"Entered DOWN trade: BUY {trade_pe}")

    time.sleep(SPOT_POLL_SEC)

write_log("=== Hybrid Breakout session ended normally ===")
print("Hybrid Breakout session ended.")
if active_trade:
  ltp = get_ltp_any(key_for(active_trade["sym"])) or active_trade["avg_buy"]
  unreal = (ltp - active_trade["avg_buy"]) * active_trade["qty"]
  entry_value = active_trade["avg_buy"] * active_trade["qty"]
  unreal_pct = (unreal / entry_value * 100) if entry_value > 0 else 0.0
  print(f"Unrealized (open)  : ₹{unreal:.2f} ({unreal_pct:.2f}%)")
print_trade_summary()

=== Hybrid Breakout Scanner (1m Close Confirm) ===
Time now: 15:17:01 IST | Regime: ROLL
Level window: 14:29 → 15:14 (excl last 3m)
PrevHigh: 25575.55 at 14:37 | UpTrig(+buf): 25580.67
PrevLow : 25448.60 at 15:09 | DnTrig(-buf): 25443.51
Last closed 1m close: 25441.50 at 15:17
Trades: 0/3 | RealizedPnL: 0.00 | PAPER=False
------------------------------------------------------------
Entered DOWN trade: BUY NIFTY26FEB25550PE
Open Trade: NIFTY26FEB25550PE | Qty:130 | Avg:201.65 | LTP:200.80 | UnrealizedPnL: ₹-110.50 (-0.42%)
Open Trade: NIFTY26FEB25550PE | Qty:130 | Avg:201.65 | LTP:200.90 | UnrealizedPnL: ₹-97.50 (-0.37%)
Open Trade: NIFTY26FEB25550PE | Qty:130 | Avg:201.65 | LTP:200.90 | UnrealizedPnL: ₹-97.50 (-0.37%)
Open Trade: NIFTY26FEB25550PE | Qty:130 | Avg:201.65 | LTP:201.35 | UnrealizedPnL: ₹-39.00 (-0.15%)
Open Trade: NIFTY26FEB25550PE | Qty:130 | Avg:201.65 | LTP:201.85 | UnrealizedPnL: ₹26.00 (0.10%)
Open Trade: NIFTY26FEB25550PE | Qty:130 | Avg:201.65 | LTP:202.20 | Unreal

SystemExit: 0

In [ ]:
# @title
# ============================================
# CELL 4 — Hybrid Breakout Auto Trader (09:50–15:30)
# Hybrid levels: Before 11:00 => ORB (09:15–09:45), After 11:00 => Rolling range
# Entry confirmation: 1-minute candle CLOSE beyond trigger + buffer
# History: seeded from market open so logic does not depend on runtime start
# Includes: paper trade option, daily loss cap, max trades/day, summary (times + duration)
# ============================================

import os, time, sys, signal
from datetime import datetime, date, timedelta, time as dtime
from zoneinfo import ZoneInfo
from collections import deque
from IPython.display import clear_output

import urllib.request
import urllib.parse

TELEGRAM_BOT_TOKEN = "8515797261:AAGf3UZsdmRCIgVy-3Y8MFLOGrkvpu6edS8"
TELEGRAM_CHAT_ID = "5782983386"

def notify_telegram(msg: str):
    if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_ID:
        return
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        data = urllib.parse.urlencode({
            "chat_id": TELEGRAM_CHAT_ID,
            "text": msg
        }).encode("utf-8")
        req = urllib.request.Request(url, data=data)
        urllib.request.urlopen(req, timeout=5)
    except Exception as e:
        # don't break trading if notification fails
        write_log(f"Telegram notify failed: {e}")

# ----------------------------
# REQUIRED VARS (expected like Cell3)
# ----------------------------
required_vars = [
    "stoploss_percent",
    "trail_start_percent",
    "trail_step_percent",
    "square_off_target_percent",
    "log_folder",
]
missing = [v for v in required_vars if v not in globals() or globals()[v] is None]
if missing:
    raise NameError(f"Missing required vars: {missing}. Define before running Cell4.")

# ----------------------------
# CONFIG
# ----------------------------
IST = ZoneInfo("Asia/Kolkata")

# Strategy run window (scanner)
START_TIME = dtime(9, 50)
END_TIME   = dtime(15, 30)
FORCE_EXIT_TIME = dtime(15, 29, 50)

# Market open
MARKET_OPEN_TIME = dtime(9, 15)

# Hybrid regime
REGIME_SWITCH_TIME = dtime(11, 0)     # before -> ORB, after -> rolling
ORB_START_TIME = dtime(9, 15)
ORB_END_TIME   = dtime(9, 45)         # ✅ as requested

# Rolling window after 11:00
ROLLING_MINUTES = 45

# Exclude last X minutes while building levels
EXCLUDE_LAST_MINUTES = 3

# Breakout buffer in percentage (e.g., 0.0015 = 0.15%)
BREAKOUT_BUFFER_PCT = 0.0002

# Risk & limits
MAX_TRADES_PER_DAY = 3
MAX_DAILY_LOSS_PCT = 0.20

# Paper trade option
PAPER_TRADE = False   # set False for LIVE

# Polling
SPOT_POLL_SEC = 1.0
MONITOR_SLEEP_SEC = 1.5

# Rearm: after a trade closes, require price to return inside band before next entry
REARM_REQUIRED = True

# --- New exit enhancements ---
TIME_STOP_MINUTES = 7

# Meaningful movement threshold in R-multiples.
# Since 1R (in %) ≈ stoploss_percent, 0.30R => profit_pct >= 0.30 * stoploss_percent
TIME_STOP_R_MULT = 0.30


# Fund usage buffer
buffer_pct = globals().get("buffer_pct", 0.02)

# Exchange prefix from Cell2
exchange_prefix = globals().get("current_prefix", "NFO")

# Enable square-off target logic
enable_square_off = globals().get("enable_square_off", True)

# Freeze limit per order
DEFAULT_MAX_FREEZE_QTY = globals().get(
    "max_freeze_qty_per_order",
    1800 if exchange_prefix == "NFO" else 600
)

# ----------------------------
# Logging
# ----------------------------
os.makedirs(log_folder, exist_ok=True)
log_file_path = os.path.join(log_folder, date.today().isoformat() + "_hybrid_breakout.txt")

def write_log(line: str):
    ts = datetime.now(IST).isoformat(timespec="seconds")
    with open(log_file_path, "a") as f:
        f.write(f"{ts} | {line}\n")

write_log("=== Hybrid Breakout session started (Cell4) ===")

def now_ist():
    return datetime.now(IST)

def combine_today(t: dtime):
    return datetime.combine(date.today(), t, tzinfo=IST)

def key_for(sym: str) -> str:
    return f"{exchange_prefix}:{sym}"

# ----------------------------
# Kite helpers
# ----------------------------
def kite_available_cash_once():
    try:
        m = kite.margins()
        return float(m["equity"]["available"]["live_balance"])
    except Exception as e:
        write_log(f"Could not fetch kite margins: {e}")
        return 0.0

def get_ltp_any(full_symbol: str):
    """full_symbol like 'NSE:NIFTY 50' or 'NFO:XYZ'"""
    try:
        d = kite.ltp([full_symbol])
        return d.get(full_symbol, {}).get("last_price")
    except Exception as e:
        write_log(f"LTP fetch failed {full_symbol}: {e}")
        return None

def get_ltps(symbols):
    if not symbols:
        return {}
    keys = [key_for(s) for s in symbols]
    try:
        data = kite.ltp(keys)
        return {s: data.get(key_for(s), {}).get("last_price") for s in symbols}
    except Exception as e:
        write_log(f"get_ltps failed: {e}")
        return {s: None for s in symbols}

TERMINAL = {"COMPLETE", "REJECTED", "CANCELLED"}

def fetch_orders_map():
    try:
        all_orders = kite.orders()
        return {o.get("order_id"): o for o in all_orders if o.get("order_id")}
    except Exception as e:
        write_log(f"fetch_orders_map failed: {e}")
        return {}

def wait_for_orders(order_ids, timeout_sec=15, poll_interval=0.35):
    order_ids = [oid for oid in order_ids if oid]
    if not order_ids:
        return {}
    start = time.time()
    last_seen = {}
    while time.time() - start < timeout_sec:
        omap = fetch_orders_map()
        for oid in order_ids:
            if oid in omap:
                last_seen[oid] = omap[oid]
        done = sum(1 for oid in order_ids if (last_seen.get(oid, {}) or {}).get("status") in TERMINAL)
        if done == len(order_ids):
            return last_seen
        time.sleep(poll_interval)
    write_log(f"wait_for_orders timeout after {timeout_sec}s; returning partial.")
    return last_seen

# ----------------------------
# LIVE order wrappers
# ----------------------------
def place_market_buy(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_BUY,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_market_sell(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_sl_sell(sym, qty, sl_price):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_SL,
        price=float(sl_price),
        trigger_price=float(sl_price)
    )

def cancel_order_fast(order_id):
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)
        return True
    except Exception as e:
        write_log(f"cancel_order_fast ignore {order_id}: {e}")
        return False

def modify_sl_order(order_id, new_price):
    try:
        kite.modify_order(
            variety=kite.VARIETY_REGULAR,
            order_id=order_id,
            price=float(new_price),
            trigger_price=float(new_price)
        )
        return True
    except Exception as e:
        write_log(f"modify_sl_order failed {order_id} -> {new_price}: {e}")
        return False

# ----------------------------
# Freeze/lot utilities
# ----------------------------
def infer_lot_size(sym: str) -> int:
    if "SENSEX" in sym.upper() or exchange_prefix == "BFO":
        return 20
    return 65

def max_units_per_order(lot_size: int) -> int:
    max_units = int(DEFAULT_MAX_FREEZE_QTY)
    lots = max(1, max_units // lot_size)
    return int(lots * lot_size)

def aligned_affordable_chunk(remaining_funds, ltp, lot_size, max_units, buffer_pct):
    if not ltp or ltp <= 0:
        return 0
    affordable_units = int(remaining_funds // (ltp * (1 + buffer_pct)))
    affordable_lots = affordable_units // lot_size
    if affordable_lots <= 0:
        return 0
    units = affordable_lots * lot_size
    return int(min(units, max_units))

def split_units_to_chunks(total_units, max_units, lot_size):
    chunks = []
    remaining = int(total_units)
    if max_units < lot_size:
        max_units = lot_size
    while remaining > 0:
        take = min(remaining, max_units)
        rem_mod = take % lot_size
        if rem_mod != 0:
            take -= rem_mod
        if take <= 0:
            take = lot_size
        chunks.append(int(take))
        remaining -= take
    return chunks

def place_split_sl_orders(sym, total_units, sl_price, lot_size):
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(total_units, max_u, lot_size)
    placed = []
    for chunk in chunks:
        try:
            oid = place_sl_sell(sym, chunk, sl_price)
            placed.append({"order_id": oid, "qty": int(chunk), "price": float(sl_price)})
            time.sleep(0.08)
        except Exception as e:
            write_log(f"ERROR placing SL chunk {sym} qty={chunk}: {e}")
    return placed

def safe_exit_real(sym, qty, sl_order_ids, lot_size, reason="exit"):
    write_log(f"[safe_exit_real] {sym} qty={qty} reason={reason}")
    # cancel SLs
    for oid in (sl_order_ids or []):
        if oid:
            cancel_order_fast(oid)
            time.sleep(0.08)
    # sell chunks
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(qty, max_u, lot_size)
    for chunk in chunks:
        try:
            place_market_sell(sym, chunk)
            time.sleep(0.08)
        except Exception as e:
            write_log(f"ERROR market sell {sym} chunk={chunk}: {e}")

# ----------------------------
# Paper broker simulator
# ----------------------------
class PaperBroker:
    def __init__(self):
        self._oid = 100000
        self.positions = {}  # sym -> {qty, w_sum, avg, sl, open}
    def _next_oid(self):
        self._oid += 1
        return str(self._oid)
    def buy_market(self, sym, qty, ltp):
        oid = self._next_oid()
        pos = self.positions.get(sym, {"qty": 0, "w_sum": 0.0, "open": False})
        pos["qty"] += int(qty)
        pos["w_sum"] += float(ltp) * int(qty)
        pos["avg"] = pos["w_sum"] / pos["qty"]
        pos["open"] = True
        self.positions[sym] = pos
        return oid, pos["avg"]
    def set_sl(self, sym, sl_price):
        if sym in self.positions:
            self.positions[sym]["sl"] = float(sl_price)
    def modify_sl(self, sym, new_sl):
        if sym in self.positions:
            self.positions[sym]["sl"] = float(new_sl)
    def exit_market(self, sym, ltp, reason="exit"):
        if sym not in self.positions or not self.positions[sym].get("open"):
            return 0.0
        pos = self.positions[sym]
        qty = int(pos["qty"])
        avg = float(pos["avg"])
        pnl = (float(ltp) - avg) * qty
        pos["open"] = False
        pos["exit_reason"] = reason
        pos["realized_pnl"] = pnl
        return pnl

paper = PaperBroker()

# ----------------------------
# Selection (from Cell2)
# ----------------------------
selected = get_selected_symbols()
if not selected:
    raise SystemExit("No symbols selected. Select in chain UI (Cell2) and re-run Cell4.")

if "symbol_meta" not in globals() or not symbol_meta:
    raise SystemExit("symbol_meta missing. Re-render selector chain (Cell2) and re-select.")

selected_ce = [s for s in selected if (symbol_meta.get(s, {}) or {}).get("instrument_type") == "CE"]
selected_pe = [s for s in selected if (symbol_meta.get(s, {}) or {}).get("instrument_type") == "PE"]
if not selected_ce or not selected_pe:
    raise SystemExit("Need at least ONE CE and ONE PE selected for UP/DOWN breakouts.")

first_meta = symbol_meta[selected[0]]
index_name = first_meta.get("index_name")

# underlying spot symbol from INDEX_CONFIG (Cell2)
spot_symbol = None
if "INDEX_CONFIG" in globals():
    for _, cfg in INDEX_CONFIG.items():
        if cfg.get("name") == index_name:
            spot_symbol = cfg.get("spot")
            break
if not spot_symbol:
    raise SystemExit("Could not determine underlying spot symbol from INDEX_CONFIG. Re-run Cell2.")

atm_data = (atm_info.get(index_name, {}) if "atm_info" in globals() else {}) or {}
atm_strike = atm_data.get("atm_strike")

def strike_distance(sym):
    if atm_strike is None:
        return 10**18
    return abs((symbol_meta.get(sym, {}) or {}).get("strike", 10**18) - atm_strike)

# nearest selected CE/PE
trade_ce = sorted(selected_ce, key=strike_distance)[0]
trade_pe = sorted(selected_pe, key=strike_distance)[0]

print("Hybrid Breakout strategy configured:")
print(f" - Underlying: {spot_symbol}")
print(f" - ORB: {ORB_START_TIME.strftime('%H:%M')}–{ORB_END_TIME.strftime('%H:%M')} (before {REGIME_SWITCH_TIME.strftime('%H:%M')})")
print(f" - After {REGIME_SWITCH_TIME.strftime('%H:%M')}: Rolling {ROLLING_MINUTES} minutes")
print(f" - UP breakout => BUY CE: {trade_ce}")
print(f" - DOWN breakout => BUY PE: {trade_pe}")
print(f" - Buffer: {BREAKOUT_BUFFER_PCT*100:.3f}% | Exclude last {EXCLUDE_LAST_MINUTES}m")
print(f" - PAPER_TRADE={PAPER_TRADE}")
print("-----------------------------------------------------")

write_log(f"Configured spot={spot_symbol}, CE={trade_ce}, PE={trade_pe}, ORB=09:15-09:45, switch=11:00, rolling={ROLLING_MINUTES}m")

# ----------------------------
# Funds + risk
# ----------------------------
initial_cash = kite_available_cash_once() if not PAPER_TRADE else 0.0
print(f"Broker Live Balance (once) = ₹{initial_cash:.2f} (ignored in paper mode)")

user_input = input("Enter fund to use for the day (e.g. 5000). Press Enter to use broker balance: ").strip()
fund = float(initial_cash) if user_input == "" else float(user_input)

usable_fund = fund * (1 - buffer_pct)
max_daily_loss = fund * MAX_DAILY_LOSS_PCT

write_log(f"Fund={fund}, usable={usable_fund}, max_daily_loss={max_daily_loss}, max_trades={MAX_TRADES_PER_DAY}")

# ----------------------------
# Trade tracking + summary
# ----------------------------
trades_taken = 0
realized_pnl = 0.0
active_trade = None
rearm_ok = True

trade_history = []  # stores trade-wise details incl times + duration

def format_hms(seconds):
    if seconds is None:
        return "NA"
    seconds = int(seconds)
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    if h > 0:
        return f"{h}h {m}m {s}s"
    if m > 0:
        return f"{m}m {s}s"
    return f"{s}s"

def print_trade_summary():
    print("\n" + "="*70)
    print("📊 DAILY TRADE SUMMARY")
    print("="*70)

    total_trades = len(trade_history)
    wins = sum(1 for t in trade_history if t["pnl"] > 0)
    losses = sum(1 for t in trade_history if t["pnl"] < 0)
    pnl_pct = (realized_pnl / fund * 100) if fund > 0 else 0.0

    print(f"Mode               : {'PAPER' if PAPER_TRADE else 'LIVE'}")
    print(f"Fund Used          : ₹{fund:.2f}")
    print(f"Max Daily Loss     : ₹{max_daily_loss:.2f} ({MAX_DAILY_LOSS_PCT*100:.1f}%)")
    print("-"*70)
    print(f"Total Trades       : {total_trades}")
    print(f"Winning Trades     : {wins}")
    print(f"Losing Trades      : {losses}")
    print("-"*70)
    print(f"Net PnL            : ₹{realized_pnl:.2f}")
    print(f"Net PnL %          : {pnl_pct:.2f}%")
    print("="*70)

    if trade_history:
        print("\n🧾 Trade-wise details:")
        for i, t in enumerate(trade_history, 1):
            bt = t.get("buy_time")
            et = t.get("exit_time")
            bt_str = bt.strftime("%H:%M:%S") if bt else "NA"
            et_str = et.strftime("%H:%M:%S") if et else "NA"
            dur_str = format_hms(t.get("duration_sec"))
            print(
                f"{i}. {t['symbol']} | {t['direction']} | Qty:{t['qty']} | "
                f"Buy:{t['avg_buy']} → Exit:{t['exit_price']} | "
                f"PnL:{t['pnl']} | {t['reason']} | "
                f"BuyTime:{bt_str} | ExitTime:{et_str} | Duration:{dur_str}"
            )
    else:
        print("No trades executed today.")

# ----------------------------
# 1-minute candle store (seed + live build)
# ----------------------------
minute_candles = deque(maxlen=20000)

_current_minute = None
_curr_candle = None

def floor_to_minute(dt: datetime):
    return dt.replace(second=0, microsecond=0)

def find_instrument_token_for_spot(spot_symbol: str):
    """
    spot_symbol like 'NSE:NIFTY 50' or 'BSE:SENSEX'
    return instrument_token if found
    """
    try:
        ex, ts = spot_symbol.split(":")
        inst = kite.instruments(ex)
        for i in inst:
            if i.get("tradingsymbol") == ts:
                return i.get("instrument_token")
    except Exception as e:
        write_log(f"Token lookup failed for {spot_symbol}: {e}")
    return None

def seed_minute_candles_from_history(start_dt: datetime, end_dt: datetime):
    token = find_instrument_token_for_spot(spot_symbol)
    if not token:
        write_log("Could not seed history: instrument_token not found. Live candles only.")
        return
    try:
        candles = kite.historical_data(token, start_dt, end_dt, "minute", continuous=False, oi=False)
        added = 0
        for c in candles:
            ts = c.get("date")
            if not ts:
                continue
            start_ts = ts.astimezone(IST)
            end_ts = start_ts + timedelta(minutes=1)
            minute_candles.append({
                "end": end_ts,
                "open": float(c["open"]),
                "high": float(c["high"]),
                "low": float(c["low"]),
                "close": float(c["close"]),
            })
            added += 1
        write_log(f"Seeded {added} minute candles from {start_dt.time()} to {end_dt.time()}.")
    except Exception as e:
        write_log(f"Seeding minute candles failed: {e}")

def update_live_minute_candle(ltp: float, ts: datetime):
    global _current_minute, _curr_candle
    if ltp is None:
        return
    m = floor_to_minute(ts)

    if _current_minute is None:
        _current_minute = m
        _curr_candle = {"end": m + timedelta(minutes=1),
                        "open": float(ltp), "high": float(ltp),
                        "low": float(ltp), "close": float(ltp)}
        return

    if m != _current_minute:
        if _curr_candle is not None:
            minute_candles.append(_curr_candle)
        _current_minute = m
        _curr_candle = {"end": m + timedelta(minutes=1),
                        "open": float(ltp), "high": float(ltp),
                        "low": float(ltp), "close": float(ltp)}
        return

    _curr_candle["high"] = max(_curr_candle["high"], float(ltp))
    _curr_candle["low"]  = min(_curr_candle["low"], float(ltp))
    _curr_candle["close"] = float(ltp)

def last_closed_candle(now_dt: datetime):
    if not minute_candles:
        return None
    cur_min = floor_to_minute(now_dt)
    for c in reversed(minute_candles):
        if c["end"] <= cur_min:
            return c
    return None

def prev_closed_candle(now_dt: datetime):
    """
    Returns the candle just before the most recent fully closed candle.
    """
    if len(minute_candles) < 2:
        return None
    cur_min = floor_to_minute(now_dt)

    # Collect closed candles (end <= current minute start)
    closed = [c for c in minute_candles if c["end"] <= cur_min]
    if len(closed) < 2:
        return None
    return closed[-2]

# Seed history from market open -> now (so runtime start doesn't matter)
seed_start = combine_today(MARKET_OPEN_TIME)
seed_end = now_ist()
if seed_end > seed_start:
    seed_minute_candles_from_history(seed_start, seed_end)

# Initialize live candle builder with current spot if possible
spot_now = get_ltp_any(spot_symbol)
if spot_now is not None:
    update_live_minute_candle(float(spot_now), now_ist())

# ----------------------------
# Hybrid level computation (with times)
# ----------------------------
def compute_levels_with_times(now_dt: datetime):
    """
    Returns:
      regime, hi, hi_time, lo, lo_time, window_start, window_end
    Times are candle-end times of the candles where high/low occurred.
    """
    if not minute_candles:
        return None, None, None, None, None, None, None

    cutoff = now_dt - timedelta(minutes=EXCLUDE_LAST_MINUTES)

    if now_dt.time() < REGIME_SWITCH_TIME:
        regime = "ORB"
        w_start = combine_today(ORB_START_TIME)
        w_end_fixed = combine_today(ORB_END_TIME)
        window_end = min(w_end_fixed, cutoff)
        window_start = w_start
    else:
        regime = "ROLL"
        window_end = cutoff
        window_start = window_end - timedelta(minutes=ROLLING_MINUTES)

    relevant = [c for c in minute_candles if (c["end"] > window_start and c["end"] <= window_end)]
    if len(relevant) < 5:
        return regime, None, None, None, None, window_start, window_end

    hi_candle = max(relevant, key=lambda x: x["high"])
    lo_candle = min(relevant, key=lambda x: x["low"])

    return (
        regime,
        hi_candle["high"], hi_candle["end"],
        lo_candle["low"],  lo_candle["end"],
        window_start, window_end
    )

# ----------------------------
# Trade open/close/monitor
# ----------------------------
def open_trade(direction, sym_to_trade):
    global active_trade, trades_taken, rearm_ok

    opt_ltp = get_ltp_any(key_for(sym_to_trade))
    if not opt_ltp:
        write_log(f"open_trade failed: no LTP for {sym_to_trade}")
        return False

    lot = infer_lot_size(sym_to_trade)
    max_u = max_units_per_order(lot)

    remaining = usable_fund
    buy_oids = []
    est_ltp = float(opt_ltp)

    # buy in chunks
    while True:
        chunk = aligned_affordable_chunk(remaining, est_ltp, lot, max_u, buffer_pct)
        if chunk <= 0:
            break

        if PAPER_TRADE:
            oid, _ = paper.buy_market(sym_to_trade, chunk, est_ltp)
            buy_oids.append(oid)
        else:
            oid = place_market_buy(sym_to_trade, chunk)
            buy_oids.append(oid)

        remaining = max(0.0, remaining - (est_ltp * int(chunk)))
        time.sleep(0.08)
        est_ltp = get_ltp_any(key_for(sym_to_trade)) or est_ltp

    if not buy_oids:
        write_log(f"open_trade: no buys possible for {sym_to_trade}")
        return False

    # compute avg buy
    if PAPER_TRADE:
        avg_buy = float(paper.positions[sym_to_trade]["avg"])
        qty = int(paper.positions[sym_to_trade]["qty"])
    else:
        status_map = wait_for_orders(buy_oids, timeout_sec=15, poll_interval=0.35)
        qty = 0
        w_sum = 0.0
        for oid in buy_oids:
            od = status_map.get(oid, {}) or {}
            if od.get("status") == "REJECTED":
                continue
            fqty = int(od.get("filled_quantity") or od.get("quantity") or 0)
            apx  = float(od.get("average_price") or 0.0)
            if fqty > 0 and apx > 0:
                qty += fqty
                w_sum += apx * fqty
        if qty <= 0:
            write_log("open_trade: qty computed as 0 from broker; aborting.")
            return False
        avg_buy = w_sum / qty

    initial_sl = round(avg_buy * (1 - stoploss_percent), 1)

    sl_entries = []
    sl_ids = []
    if PAPER_TRADE:
        paper.set_sl(sym_to_trade, initial_sl)
    else:
        sl_entries = place_split_sl_orders(sym_to_trade, qty, initial_sl, lot)
        sl_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]

    active_trade = {
        "direction": direction,
        "sym": sym_to_trade,
        "qty": int(qty),
        "lot": int(lot),
        "avg_buy": float(avg_buy),
        "current_sl": float(initial_sl),
        "sl_entries": sl_entries,
        "sl_order_ids": sl_ids,
        "opened_at": now_ist(),   # buy time
        "trail_active": False,               # becomes True once profit_pct >= trail_start_percent
        "last_reversal_candle_end": None,    # ensures reversal check runs once per candle close

    }

    trades_taken += 1
    rearm_ok = False

    notify_telegram(
        f"✅ TRADE OPEN | {direction} | {sym_to_trade} | "
        f"Qty={qty} | Avg={avg_buy:.2f} | SL={initial_sl:.2f} | Time={now_ist().strftime('%H:%M:%S')}")

    write_log(f"TRADE_OPEN {direction} sym={sym_to_trade} qty={qty} avg={avg_buy:.2f} SL={initial_sl} trades={trades_taken}")

    return True

def close_trade(reason, ltp):
    global active_trade, realized_pnl, trade_history

    sym = active_trade["sym"]
    qty = active_trade["qty"]
    avg_buy = active_trade["avg_buy"]
    direction = active_trade["direction"]

    buy_time = active_trade.get("opened_at")
    exit_time = now_ist()
    duration_sec = int((exit_time - buy_time).total_seconds()) if buy_time else None

    if PAPER_TRADE:
        pnl = paper.exit_market(sym, ltp, reason=reason)
    else:
        safe_exit_real(sym, qty, active_trade.get("sl_order_ids", []), active_trade["lot"], reason=reason)
        pnl = (ltp - avg_buy) * qty  # best-effort estimate

    realized_pnl += pnl

    trade_history.append({
        "symbol": sym,
        "direction": direction,
        "qty": qty,
        "avg_buy": round(avg_buy, 2),
        "exit_price": round(float(ltp), 2),
        "pnl": round(float(pnl), 2),
        "reason": reason,
        "buy_time": buy_time,
        "exit_time": exit_time,
        "duration_sec": duration_sec,
    })
    notify_telegram(
        f"❌ TRADE CLOSE | {reason} | {sym} | "
        f"Qty={qty} | Exit={ltp:.2f} | PnL={pnl:.2f} | "
        f"Realized={realized_pnl:.2f} | Time={now_ist().strftime('%H:%M:%S')}")

    write_log(f"TRADE_CLOSE {('PAPER' if PAPER_TRADE else 'REAL')} {reason} sym={sym} pnl={pnl:.2f} realized={realized_pnl:.2f}")
    active_trade = None

def monitor_active_trade():
    """
    Monitor SL/trailing/target more frequently than candle closes.
    Returns True if still open, False if closed.
    """
    global active_trade

    if not active_trade:
        return False

    sym = active_trade["sym"]
    avg_buy = active_trade["avg_buy"]
    current_sl = active_trade["current_sl"]

    ltp = get_ltp_any(key_for(sym))
    if not ltp:
        return True

    profit_pct = (ltp - avg_buy) / avg_buy if avg_buy else 0.0

    # ----------------------------
    # (1) TIME STOP: exit if no meaningful movement within 7 minutes
    # Meaningful movement defined as reaching TIME_STOP_R_MULT * 1R
    # and 1R (as %) is approximately stoploss_percent
    # ----------------------------
    elapsed_sec = (now_ist() - active_trade["opened_at"]).total_seconds()
    meaningful_profit_pct = TIME_STOP_R_MULT * stoploss_percent

    if elapsed_sec >= TIME_STOP_MINUTES * 60 and profit_pct < meaningful_profit_pct:
        close_trade("TIME_STOP", ltp)
        return False

    # SL hit
    if ltp <= current_sl:
        close_trade("SL", ltp)
        return False

    # Trailing SL
    if profit_pct >= trail_start_percent:
        active_trade["trail_active"] = True
        trail_step_value = avg_buy * trail_step_percent
        new_sl = round(ltp - trail_step_value, 1)
        if new_sl > current_sl:
            if PAPER_TRADE:
                paper.modify_sl(sym, new_sl)
            else:
                ok = 0
                for e in (active_trade.get("sl_entries") or []):
                    oid = e.get("order_id")
                    if oid and modify_sl_order(oid, new_sl):
                        ok += 1
                        e["price"] = float(new_sl)
                        time.sleep(0.08)
                write_log(f"TRAIL_SL REAL sym={sym} new_sl={new_sl} modified={ok}")
            active_trade["current_sl"] = float(new_sl)
            write_log(f"TRAIL_SL sym={sym} -> {new_sl}")

    # ----------------------------
    # (2) REVERSAL EXIT (underlying candle): only after trailing starts
    # UP trade: 1m close < prev candle low
    # DOWN trade: 1m close > prev candle high
    # ----------------------------
    if active_trade.get("trail_active", False):
        lc = last_closed_candle(now_ist())
        pc = prev_closed_candle(now_ist())

        if lc and pc:
            # ensure we check only once per newly closed candle
            if active_trade.get("last_reversal_candle_end") != lc["end"]:
                active_trade["last_reversal_candle_end"] = lc["end"]

                close_px = lc["close"]
                if active_trade["direction"] == "UP":
                    # exit if reversal against UP move
                    if close_px < pc["low"]:
                        close_trade("REV_EXIT", ltp)
                        return False

                elif active_trade["direction"] == "DOWN":
                    # exit if reversal against DOWN move
                    if close_px > pc["high"]:
                        close_trade("REV_EXIT", ltp)
                        return False
    # Target square-off
    if enable_square_off and profit_pct >= square_off_target_percent:
        close_trade("TARGET", ltp)
        return False

    return True

# ----------------------------
# Emergency stop
# ----------------------------
def emergency_exit(signum=None, frame=None):
    print("\nEMERGENCY STOP received.")
    write_log("EMERGENCY STOP received.")
    if active_trade:
        sym = active_trade["sym"]
        ltp = get_ltp_any(key_for(sym)) or active_trade["avg_buy"]
        close_trade("EMERGENCY", ltp)
    write_log("=== Hybrid Breakout session ended by emergency ===")
    print_trade_summary()
    sys.exit(0)

signal.signal(signal.SIGINT, emergency_exit)

# ----------------------------
# Main loop: hybrid levels + 1m close confirmation
# ----------------------------
def in_trading_window(t: datetime):
    return START_TIME <= t.time() <= END_TIME

def past_force_exit(t: datetime):
    return t.time() >= FORCE_EXIT_TIME

print("Starting Hybrid Breakout loop... (Ctrl+C for emergency stop)")
write_log("Main loop started.")

# wait until START_TIME
while now_ist().time() < START_TIME:
    clear_output(wait=True)
    print("Waiting for start time 09:50 IST...")
    time.sleep(5)

last_signal_candle_end = None

while True:
    t = now_ist()

    # end of day
    if t.time() > END_TIME:
        write_log("End time reached. Stopping.")
        break

    # update live candle builder from spot LTP
    spot = get_ltp_any(spot_symbol)
    if spot is not None:
        update_live_minute_candle(float(spot), t)

    # If trade open: risk check + monitor
    if active_trade:
        # daily loss check includes unrealized
        ltp = get_ltp_any(key_for(active_trade["sym"])) or active_trade["avg_buy"]
        unreal = (ltp - active_trade["avg_buy"]) * active_trade["qty"]
        # --- NEW: compute unrealized pnl % for display (no behavior change) ---
        entry_value = active_trade["avg_buy"] * active_trade["qty"]
        unreal_pct = (unreal / entry_value * 100) if entry_value > 0 else 0.0

        # --- NEW: print unrealized pnl line (keeps existing output, adds one extra line) ---
        print(f"Open Trade: {active_trade['sym']} | Qty:{active_trade['qty']} | "
              f"Avg:{active_trade['avg_buy']:.2f} | LTP:{ltp:.2f} | "
              f"UnrealizedPnL: ₹{unreal:.2f} ({unreal_pct:.2f}%)")
        if (realized_pnl + unreal) <= -max_daily_loss:
            write_log("Max daily loss hit (realized+unrealized). Forcing exit & stop.")
            close_trade("MAX_LOSS", ltp)
            break

        # force exit near market close
        if past_force_exit(t):
            write_log("Force exit time reached. Closing open trade.")
            close_trade("TIME_EXIT", ltp)
            break

        monitor_active_trade()
        time.sleep(MONITOR_SLEEP_SEC)
        continue

    # no active trade: stop if max daily loss reached (realized)
    if realized_pnl <= -max_daily_loss:
        write_log("Max daily loss hit (realized). Stopping.")
        break

    # no new entries after max trades
    if trades_taken >= MAX_TRADES_PER_DAY:
        clear_output(wait=True)
        print(f"[{t.strftime('%H:%M:%S')}] Max trades reached ({trades_taken}). No new entries.")
        print(f"RealizedPnL={realized_pnl:.2f}")
        time.sleep(SPOT_POLL_SEC)
        continue

    # operate only inside trading window
    if not in_trading_window(t):
        time.sleep(SPOT_POLL_SEC)
        continue

    # 1-minute close confirmation: only process when we have a NEW closed candle
    lc = last_closed_candle(t)
    if lc is None:
        time.sleep(SPOT_POLL_SEC)
        continue

    if last_signal_candle_end == lc["end"]:
        time.sleep(SPOT_POLL_SEC)
        continue

    close_px = lc["close"]
    close_time = lc["end"]

    regime, prev_high, prev_high_time, prev_low, prev_low_time, w_start, w_end = compute_levels_with_times(t)

    # mark processed candle regardless (prevents spam)
    last_signal_candle_end = lc["end"]

    if prev_high is None or prev_low is None:
        time.sleep(SPOT_POLL_SEC)
        continue

    up_level = prev_high * (1 + BREAKOUT_BUFFER_PCT)
    dn_level = prev_low  * (1 - BREAKOUT_BUFFER_PCT)

    # Rearm check (use close confirmation band)
    if REARM_REQUIRED and not rearm_ok:
        if dn_level < close_px < up_level:
            rearm_ok = True
            write_log("Re-armed for next trade (close back inside band).")
        time.sleep(SPOT_POLL_SEC)
        continue

    # Print scanner state including high/low times
    clear_output(wait=True)
    print("=== Hybrid Breakout Scanner (1m Close Confirm) ===")
    print(f"Time now: {t.strftime('%H:%M:%S')} IST | Regime: {regime}")
    print(f"Level window: {w_start.strftime('%H:%M')} → {w_end.strftime('%H:%M')} (excl last {EXCLUDE_LAST_MINUTES}m)")
    print(f"PrevHigh: {prev_high:.2f} at {prev_high_time.strftime('%H:%M')} | UpTrig(+buf): {up_level:.2f}")
    print(f"PrevLow : {prev_low:.2f} at {prev_low_time.strftime('%H:%M')} | DnTrig(-buf): {dn_level:.2f}")
    print(f"Last closed 1m close: {close_px:.2f} at {close_time.strftime('%H:%M')}")
    print(f"Trades: {trades_taken}/{MAX_TRADES_PER_DAY} | RealizedPnL: {realized_pnl:.2f} | PAPER={PAPER_TRADE}")
    print("-"*60)

    # Decide entry ONLY on candle close confirmation
    direction = None
    if close_px > up_level:
        direction = "UP"
    elif close_px < dn_level:
        direction = "DOWN"

    if direction == "UP":
        write_log(f"BREAKOUT UP (1m close) close={close_px:.2f} > {up_level:.2f}. Enter CE={trade_ce}")
        if open_trade("UP", trade_ce):
            print(f"Entered UP trade: BUY {trade_ce}")
    elif direction == "DOWN":
        write_log(f"BREAKOUT DOWN (1m close) close={close_px:.2f} < {dn_level:.2f}. Enter PE={trade_pe}")
        if open_trade("DOWN", trade_pe):
            print(f"Entered DOWN trade: BUY {trade_pe}")

    time.sleep(SPOT_POLL_SEC)

write_log("=== Hybrid Breakout session ended normally ===")
print("Hybrid Breakout session ended.")
if active_trade:
  ltp = get_ltp_any(key_for(active_trade["sym"])) or active_trade["avg_buy"]
  unreal = (ltp - active_trade["avg_buy"]) * active_trade["qty"]
  entry_value = active_trade["avg_buy"] * active_trade["qty"]
  unreal_pct = (unreal / entry_value * 100) if entry_value > 0 else 0.0
  print(f"Unrealized (open)  : ₹{unreal:.2f} ({unreal_pct:.2f}%)")
print_trade_summary()

Hybrid Breakout strategy configured:
 - Underlying: NSE:NIFTY 50
 - ORB: 09:15–09:45 (before 11:00)
 - After 11:00: Rolling 45 minutes
 - UP breakout => BUY CE: NIFTY26FEB25450CE
 - DOWN breakout => BUY PE: NIFTY26FEB25350PE
 - Buffer: 0.020% | Exclude last 3m
 - PAPER_TRADE=False
-----------------------------------------------------
Broker Live Balance (once) = ₹41851.80 (ignored in paper mode)
Enter fund to use for the day (e.g. 5000). Press Enter to use broker balance: 
Starting Hybrid Breakout loop... (Ctrl+C for emergency stop)
Hybrid Breakout session ended.

📊 DAILY TRADE SUMMARY
Mode               : LIVE
Fund Used          : ₹41851.80
Max Daily Loss     : ₹8370.36 (20.0%)
----------------------------------------------------------------------
Total Trades       : 0
Winning Trades     : 0
Losing Trades      : 0
----------------------------------------------------------------------
Net PnL            : ₹0.00
Net PnL %          : 0.00%
No trades executed today.
